# Reading data

In [1]:
import numpy as np
import os
from collections import OrderedDict
import matplotlib.pyplot as plt
from scipy.misc import imsave
from resizeimage import resizeimage
from PIL import Image

# from objectives.VoObjectives import rmse, vo_loss
from models.AbstractModel import AbstractModel
from dataset.DataGenerationStrategy import OpticalFlowGenerationStrategy
from dataset.Sequence import Sequence
import robot as rb


Robotics Toolbox for Python
Based on Matlab Toolbox Version 7  April-2002

What's new.
  Readme      - New features and enhancements in this version.

Homogeneous transformations
  eul2tr      - Euler angle to transform 
  oa2tr       - orientation and approach vector to transform 
  rotx        - transform for rotation about X-axis 
  roty        - transform for rotation about Y-axis 
  rotz        - transform for rotation about Z-axis 
  rpy2tr      - roll/pitch/yaw angles to transform 
  tr2eul      - transform to Euler angles 
  tr2rot      - transform to rotation submatrix
  tr2rpy      - transform to roll/pitch/yaw angles
  transl      - set or extract the translational component of a transform 
  trnorm      - normalize a transform 
  
Quaternions
  /           - divide quaternion by quaternion or scalar
  *           - multiply quaternion by a quaternion or vector
  inv         - invert a quaternion 
  norm        - norm of a quaternion 
  plot        - display a quaternion as

Using TensorFlow backend.


In [2]:
data_set_dir = 'D:/KITTI/dataset/'

kitti_train_dirs = ['00', '01', '02', '03', '04', '05', '06', '07']
kitti_test_dirs = ['08', '09', '10']

input_width = 155
input_height = 47

#input_width = 310
#input_height = 94

input_channel = 1
kitti_is_grayscale = True

training_seqs = OrderedDict()
test_seqs = OrderedDict()

name = 'kitti'

In [3]:
def read_data():

        subdir = 'disparity'

        
        for dir in kitti_train_dirs:
            seq_dir = os.path.normpath(os.path.join(data_set_dir, subdir, dir))
            training_seqs[dir] = Sequence(seq_dir,
                                               extension='png',
                                               label_file=os.path.join(seq_dir, dir  + '.txt'),
                                               dir=dir,
                                               is_grayscale=kitti_is_grayscale,
                                               name='Kitti_train/' + dir)

        for dir in kitti_test_dirs:
            seq_dir = os.path.normpath(os.path.join(data_set_dir, subdir, dir))
            print(dir)
            
            print(os.path.join(seq_dir, subdir))
            test_seqs[dir] = Sequence(seq_dir,
                                           extension='png',
                                           label_file=os.path.join(seq_dir, dir  + '.txt'),
                                           dir=dir,
                                           is_grayscale=kitti_is_grayscale,
                                           name='Kitti_test/' + dir)
            
def print_info():

        print('--------------------------')
        print('------Dataset Info--------')
        print('Dataset Name: {}'.format(name))
        print('Number of Training dirs: {}'.format(len(training_seqs)))
        print('Training dirs:')
        for directory in training_seqs:
            curr_sequence = training_seqs[directory]
            print(directory,
                  curr_sequence.sequence_dir,
                  'Num imgs: {}'.format(curr_sequence.get_num_imgs()),
                  'Num label: {}'.format(curr_sequence.get_num_label()))

        print('Number of Test dirs: {}'.format(len(test_seqs)))
        print('Test dirs:')
        for directory in test_seqs:
            curr_sequence = test_seqs[directory]
            print(directory,
                  curr_sequence.sequence_dir,
                  'Num imgs: {}'.format(curr_sequence.get_num_imgs()),
    'Num label: {}'.format(curr_sequence.get_num_label()))        

In [4]:
read_data()
print_info()

08
D:\KITTI\dataset\disparity\08\disparity
09
D:\KITTI\dataset\disparity\09\disparity
10
D:\KITTI\dataset\disparity\10\disparity
--------------------------
------Dataset Info--------
Dataset Name: kitti
Number of Training dirs: 8
Training dirs:
00 D:\KITTI\dataset\disparity\00 Num imgs: 4541 Num label: 4541
01 D:\KITTI\dataset\disparity\01 Num imgs: 1101 Num label: 1101
02 D:\KITTI\dataset\disparity\02 Num imgs: 4661 Num label: 4661
03 D:\KITTI\dataset\disparity\03 Num imgs: 801 Num label: 801
04 D:\KITTI\dataset\disparity\04 Num imgs: 271 Num label: 271
05 D:\KITTI\dataset\disparity\05 Num imgs: 2761 Num label: 2761
06 D:\KITTI\dataset\disparity\06 Num imgs: 1101 Num label: 1101
07 D:\KITTI\dataset\disparity\07 Num imgs: 1101 Num label: 1101
Number of Test dirs: 3
Test dirs:
08 D:\KITTI\dataset\disparity\08 Num imgs: 4071 Num label: 4071
09 D:\KITTI\dataset\disparity\09 Num imgs: 1591 Num label: 1591
10 D:\KITTI\dataset\disparity\10 Num imgs: 1201 Num label: 1201


## Read and transform training pose

In [5]:
cont = 0
label_training_all = {}
for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   
        
        t_aux = matrix[0:3, 3]        
        t = np.zeros((1,3))
        
        for i in range(0,3):
            t[0, i] = t_aux[i]
            cont += 1
            
        rpy = rb.tr2rpy(matrix)
        label_aux.append(np.concatenate((rpy, t),1))
        
    label_training_all[name] = label_aux[1:]

print(len(label_training_all))
print(set(label_training_all))

8
{'05', '04', '01', '00', '03', '06', '07', '02'}


In [6]:
for element in label_training_all:
    print("Size of sequence %s: %s"%(element, len(label_training_all[element])))

Size of sequence 01: 1100
Size of sequence 06: 1100
Size of sequence 05: 2760
Size of sequence 07: 1100
Size of sequence 04: 270
Size of sequence 00: 4540
Size of sequence 03: 800
Size of sequence 02: 4660


## Read and normalize training images 

In [7]:
dataset_training_all_l = {}
dataset_training_all_r = {}

for directory in training_seqs:
    curr_sequence = training_seqs[directory]
    paths = curr_sequence.get_image_paths()
    name = curr_sequence.get_dir()
    num_images = curr_sequence.get_num_imgs()
    img_aux = []
    cont = 0
    print("Sequence: ", name)    
    for path in paths:
        img = Image.open(path)        
        img = img.resize((input_width, input_height))
        
        img_new = np.array(img.getdata()).reshape(img.size[1], img.size[0])
        img_new = img_new / 255.0        
        img_aux.append(img_new)
        
        del(img_new)
        del(img)

        print("%s \r"%cont, end="", flush=True)    
        cont += 1
        
    print("List length: ", len(img_aux))
    print("Right list length: ", len(img_aux[1:]))
    print("Left list length: ", len(img_aux[:-1]))
    print("")
    
    dataset_training_all_r[name] = img_aux[1:]
    dataset_training_all_l[name] = img_aux[:-1]
    
    del(img_aux)
    

Sequence:  00
List length:  4541
Right list length:  4540
Left list length:  4540

Sequence:  01
List length:  1101
Right list length:  1100
Left list length:  1100

Sequence:  02
List length:  4661
Right list length:  4660
Left list length:  4660

Sequence:  03
List length:  801
Right list length:  800
Left list length:  800

Sequence:  04
List length:  271
Right list length:  270
Left list length:  270

Sequence:  05
List length:  2761
Right list length:  2760
Left list length:  2760

Sequence:  06
List length:  1101
Right list length:  1100
Left list length:  1100

Sequence:  07
List length:  1101
Right list length:  1100
Left list length:  1100



In [8]:
print("Training right: ")
print(len(dataset_training_all_r))
print(set(dataset_training_all_r))

for element in dataset_training_all_r:
    img = dataset_training_all_r[element][0]
    print("Sequence %s: Size = %s and dim = %s."%(element, len(dataset_training_all_r[element]), 
                                                  np.shape(img)))
    imsave("%s.jpg"%element, img.reshape(input_height, input_width))
    
print(" ")
print("Training left: ")

print(len(dataset_training_all_l))
print(set(dataset_training_all_l))

for element in dataset_training_all_l:
    img = dataset_training_all_l[element][0]
    print("Sequence %s: Size = %s and dim = %s."%(element, len(dataset_training_all_l[element]), 
                                                  np.shape(img)))
    imsave("%s.jpg"%element, img.reshape(input_height, input_width))

Training right: 
8
{'05', '04', '01', '00', '03', '06', '07', '02'}
Sequence 01: Size = 1100 and dim = (47, 155).
Sequence 06: Size = 1100 and dim = (47, 155).
Sequence 05: Size = 2760 and dim = (47, 155).
Sequence 07: Size = 1100 and dim = (47, 155).
Sequence 04: Size = 270 and dim = (47, 155).
Sequence 00: Size = 4540 and dim = (47, 155).
Sequence 03: Size = 800 and dim = (47, 155).
Sequence 02: Size = 4660 and dim = (47, 155).
 
Training left: 
8
{'05', '04', '01', '00', '03', '06', '07', '02'}
Sequence 01: Size = 1100 and dim = (47, 155).
Sequence 06: Size = 1100 and dim = (47, 155).
Sequence 05: Size = 2760 and dim = (47, 155).
Sequence 07: Size = 1100 and dim = (47, 155).
Sequence 04: Size = 270 and dim = (47, 155).
Sequence 00: Size = 4540 and dim = (47, 155).
Sequence 03: Size = 800 and dim = (47, 155).
Sequence 02: Size = 4660 and dim = (47, 155).


## Read and test pose

In [9]:
cont = 0
label_test_all = {}
for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   
        
        t_aux = matrix[0:3, 3]        
        t = np.zeros((1,3))
        
        for i in range(0,3):
            t[0, i] = t_aux[i]
            cont += 1
            
        rpy = rb.tr2rpy(matrix)
        label_aux.append(np.concatenate((rpy, t),1))
        
    label_test_all[name] = label_aux[1:]

print(len(label_test_all))
print(set(label_test_all))

3
{'09', '08', '10'}


In [10]:
for element in label_test_all:
    print("Size of sequence %s: %s"%(element, len(label_test_all[element])))

Size of sequence 10: 1200
Size of sequence 08: 4070
Size of sequence 09: 1590


## Read and normalize test images 

In [11]:
dataset_test_all_r = {}
dataset_test_all_l = {}

for directory in test_seqs:
    curr_sequence = test_seqs[directory]
    paths = curr_sequence.get_image_paths()
    name = curr_sequence.get_dir()
    num_images = curr_sequence.get_num_imgs()
    img_aux = []
    cont = 0
    print("Sequence: ", name)
    
    for path in paths:
        img = Image.open(path)        
        img = img.resize((input_width, input_height))
        
        img_new = np.array(img.getdata()).reshape(img.size[1], img.size[0])
        img_new = img_new / 255.0        
        img_aux.append(img_new)
        
        del(img_new)
        del(img)
        percentage = round((cont/num_images)*100,2)
        
        print("%s \r"%cont, end="", flush=True)    
        cont += 1
        
    print("List length: ", len(img_aux))
    print("Right list length: ", len(img_aux[1:]))
    print("Left list length: ", len(img_aux[:-1]))
    print("")
    
    dataset_test_all_r[name] = img_aux[1:]
    dataset_test_all_l[name] = img_aux[:-1]
    
    del(img_aux)
    

Sequence:  08
List length:  4071
Right list length:  4070
Left list length:  4070

Sequence:  09
List length:  1591
Right list length:  1590
Left list length:  1590

Sequence:  10
List length:  1201
Right list length:  1200
Left list length:  1200



In [12]:
print("Test right: ")
print(len(dataset_test_all_r))
print(set(dataset_test_all_r))

for element in dataset_test_all_r:
    img = dataset_test_all_r[element][0]
    print("Sequence %s: Size = %s and dim = %s."%(element, len(dataset_test_all_r[element]), np.shape(img)))
    imsave("%s.jpg"%element, img.reshape(input_height, input_width))
          
print(" ")
print("Test left: ")

print(len(dataset_test_all_l))
print(set(dataset_test_all_l))

for element in dataset_test_all_l:
    img = dataset_test_all_l[element][0]
    print("Sequence %s: Size = %s and dim = %s."%(element, len(dataset_test_all_l[element]), np.shape(img)))
    imsave("%s.jpg"%element, img.reshape(input_height, input_width))

Test right: 
3
{'09', '08', '10'}
Sequence 10: Size = 1200 and dim = (47, 155).
Sequence 08: Size = 4070 and dim = (47, 155).
Sequence 09: Size = 1590 and dim = (47, 155).
 
Test left: 
3
{'09', '08', '10'}
Sequence 10: Size = 1200 and dim = (47, 155).
Sequence 08: Size = 4070 and dim = (47, 155).
Sequence 09: Size = 1590 and dim = (47, 155).


In [13]:
training_r = []
training_l = []
training_poses = []

test_r = []
test_l = []
test_poses = []

for sequence in kitti_train_dirs:
    
    for img in dataset_training_all_r[sequence]:
        training_r.append(img)        
    for img in dataset_training_all_l[sequence]:
        training_l.append(img)   
    for pose in label_training_all[sequence]:
        training_poses.append(pose)
        
        #training_r.append(dataset_training_all_r[sequence])
        #training_l.append(dataset_training_all_l[sequence][:])
        #training_poses.append(label_training_all[sequence][:])
        
#del(dataset_training_all_r)
#del(dataset_training_all_l)
#del(label_training_all)

print("Training > size right: %s, size left: %s, size label: %s"%
      (len(training_r), len(training_l), len(training_poses)))

#r = training_poses
#print(len(r))
print(" ")

for sequence in kitti_test_dirs:    
        for img in dataset_test_all_r[sequence]:
            test_r.append(img)        
        for img in dataset_test_all_l[sequence]:
            test_l.append(img)   
        for pose in label_test_all[sequence]:
            test_poses.append(pose)
            
#del(dataset_test_all_r)
#del(dataset_test_all_l)
#del(label_test_all)

    #test_r.append(dataset_test_all_r[sequence])
    #test_l.append(dataset_test_all_l[sequence])
    #test_poses.append(label_test_all[sequence])
    
print("Test > Size right: %s, size left: %s, size label: %s"%
      (len(test_r), len(test_l), len(test_poses)))

#l = test_poses['all']
#print(len(l))
print(" ")


Training > size right: 16330, size left: 16330, size label: 16330
 
Test > Size right: 6860, size left: 6860, size label: 6860
 


# Definindo a arquitetura

In [14]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

learning_rate = 0.0001
epochs = 100

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(LeakyReLU(alpha=0.6))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(LeakyReLU(alpha=0.6))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(LeakyReLU(alpha=0.6))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(LeakyReLU(alpha=0.6))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(2048, kernel_initializer='normal')(abs_diff)
activation1 = LeakyReLU(alpha=0.6)(dense_layer)
#dense_layer1 = Dense(2048, kernel_initializer='normal')(activation1)
#activation2 = LeakyReLU(alpha=0.5)(dense_layer1)
dense_layer2 = Dense(2048, kernel_initializer='normal')(activation1)
activation3 = LeakyReLU(alpha=0.2)(dense_layer2)
output = Dense(6, name='predictions', kernel_initializer='normal')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 8704)         92672       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 8704)         0           sequential_1[1][0]               
          

### Preparando a entrada para o padrão Tensorflow

In [14]:
training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

print(np.shape(training_r))

test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
print(np.shape(test_r))

(16330, 47, 155, 1)
(6860, 47, 155, 1)


## Diferente criação das poses

### Training pose

In [15]:
cont = 0
label_training_0 = {}
label_training_1 = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   
        label_aux.append(matrix)
        
    label_training_0[name] = label_aux[:-1]
    label_training_1[name] = label_aux[1:]


## Transformation

In [16]:
T_train = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_training_0[name], label_training_1[name]):
        transformation = pose_0.dot(np.linalg.inv(pose_1))
        t_aux = transformation[0:3, 3]        
        t = np.zeros((1,3))
        
        for i in range(0,3):
            t[0, i] = t_aux[i]
            cont += 1
            
        rpy = rb.tr2rpy(transformation)
        
        T_aux.append(np.concatenate((rpy, t),1))
    
        T_train[name] = T_aux    

In [17]:
training_poses_transformed = []

for sequence in kitti_train_dirs:
  
    for pose in T_train[sequence]:
        training_poses_transformed.append(pose)

print("Training > size right: %s, size left: %s, size label: %s"%
      (len(training_r), len(training_l), len(training_poses)))

Training > size right: 16330, size left: 16330, size label: 16330


### Test pose

In [18]:
cont = 0
label_test_0 = {}
label_test_1 = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))
        label_aux.append(matrix)
        
        
    label_test_0[name] = label_aux[:-1]
    label_test_1[name] = label_aux[1:]


### Transformation

In [19]:
T_test = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_test_0[name], label_test_1[name]):
        transformation = pose_0.dot(np.linalg.inv(pose_1))
        t_aux = transformation[0:3, 3]        
        t = np.zeros((1,3))
        
        for i in range(0,3):
            t[0, i] = t_aux[i]
            cont += 1
            
        rpy = rb.tr2rpy(transformation)
        
        T_aux.append(np.concatenate((rpy, t),1))
    
    T_test[name] = T_aux

In [20]:
test_poses_transformed = []

for sequence in kitti_test_dirs:    
        for pose in T_test[sequence]:
            test_poses_transformed.append(pose)
    
print("Test > Size right: %s, size left: %s, size label: %s"%
      (len(test_r), len(test_l), len(test_poses)))

Test > Size right: 6860, size left: 6860, size label: 6860


### Preparando a entrada para o padrão Tensorflow

In [21]:
y_training = np.reshape(training_poses_transformed, (len(training_poses), 6))
y_test = np.reshape(test_poses_transformed, (len(test_poses), 6))
print(np.shape(y_training))

(16330, 6)


In [23]:
def vo_loss2(y_true, y_pred):
    loss_value = vo_loss(y_true, y_pred)*100    
    return loss_value

learning_rate = 0.0001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss], optimizer=opt, metrics=[rmse, 'acc'])

In [24]:
num_epochs = 200

### Preparando os checkpoints

In [34]:
model_path = "D:/KITTI/Graph/DepthSiameseVO/model/1/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthSiameseVO/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/200
16330/16330 [==============================] - 50s 3ms/step - loss: 4.1864 - rmse: 3.3449 - acc: 0.2222 - val_loss: 2.2808 - val_rmse: 1.7376 - val_acc: 0.2407

Epoch 00001: val_loss improved from inf to 2.28078, saving model to D:/KITTI/Graph/DepthSiameseVO/model/1/weights.best.by.val.loss.hdf5

Epoch 00001: acc improved from -inf to 0.22217, saving model to D:/KITTI/Graph/DepthSiameseVO/model/1/weights.best.by.accuracy.hdf5
Epoch 2/200
16330/16330 [==============================] - 49s 3ms/step - loss: 2.1557 - rmse: 3.1481 - acc: 0.2715 - val_loss: 1.8118 - val_rmse: 1.5740 - val_acc: 0.2908

Epoch 00002: val_loss improved from 2.28078 to 1.81184, saving model to D:/KITTI/Graph/DepthSiameseVO/model/1/weights.best.by.val.loss.hdf5

Epoch 00002: acc improved from 0.22217 to 0.27152, saving model to D:/KITTI/Graph/DepthSiameseVO/model/1/weights.best.by.accuracy.hdf5
Epoch 3/200
16330/16330 [==============================] - 4

KeyboardInterrupt: 

## Mudar a função de perda

In [35]:
def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]* 0.3
    mean_rot = mean[:, 0:3] * 150
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

learning_rate = 0.0001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod], optimizer=opt, metrics=[rmse, 'acc'])



### Preparando os checkpoints

In [36]:
model_path = "D:/KITTI/Graph/DepthSiameseVO/model/2/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthSiameseVO/2/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthSiameseVO/model/2/
Train on 16330 samples, validate on 6860 samples
Epoch 1/200
16330/16330 [==============================] - 50s 3ms/step - loss: 0.6345 - rmse: 1.2921 - acc: 0.6245 - val_loss: 0.9832 - val_rmse: 1.8217 - val_acc: 0.3277

Epoch 00001: val_loss improved from inf to 0.98322, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.val.loss.hdf5

Epoch 00001: acc improved from -inf to 0.62449, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 2/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.5798 - rmse: 1.2189 - acc: 0.6306 - val_loss: 1.0136 - val_rmse: 1.7706 - val_acc: 0.3292

Epoch 00002: val_loss did not improve from 0.98322

Epoch 00002: acc improved from 0.62449 to 0.63056, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 3/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.5556 - rmse: 1.1949 - acc: 0


Epoch 00045: acc did not improve from 0.75585
Epoch 46/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.2692 - rmse: 0.6071 - acc: 0.7583 - val_loss: 0.9025 - val_rmse: 1.7555 - val_acc: 0.3452

Epoch 00046: val_loss did not improve from 0.85409

Epoch 00046: acc improved from 0.75585 to 0.75830, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 47/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.2695 - rmse: 0.6023 - acc: 0.7560 - val_loss: 0.8630 - val_rmse: 1.6775 - val_acc: 0.3347

Epoch 00047: val_loss did not improve from 0.85409

Epoch 00047: acc did not improve from 0.75830
Epoch 48/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.2654 - rmse: 0.5893 - acc: 0.7604 - val_loss: 0.8659 - val_rmse: 1.6824 - val_acc: 0.3509

Epoch 00048: val_loss did not improve from 0.85409

Epoch 00048: acc improved from 0.75830 to 0.76038, saving model to D:/KITTI/Graph/DepthSiamese

Epoch 70/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.2133 - rmse: 0.4895 - acc: 0.7979 - val_loss: 0.8481 - val_rmse: 1.6649 - val_acc: 0.3380

Epoch 00070: val_loss did not improve from 0.84662

Epoch 00070: acc improved from 0.79173 to 0.79786, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 71/200
16330/16330 [==============================] - 50s 3ms/step - loss: 0.2080 - rmse: 0.4789 - acc: 0.7957 - val_loss: 0.8722 - val_rmse: 1.7328 - val_acc: 0.3475

Epoch 00071: val_loss did not improve from 0.84662

Epoch 00071: acc did not improve from 0.79786
Epoch 72/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.2086 - rmse: 0.4842 - acc: 0.7977 - val_loss: 0.8620 - val_rmse: 1.6938 - val_acc: 0.3431

Epoch 00072: val_loss did not improve from 0.84662

Epoch 00072: acc did not improve from 0.79786
Epoch 73/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2069 - rmse:

Epoch 96/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1777 - rmse: 0.4290 - acc: 0.8126 - val_loss: 0.8410 - val_rmse: 1.6629 - val_acc: 0.3430

Epoch 00096: val_loss did not improve from 0.83156

Epoch 00096: acc did not improve from 0.81690
Epoch 97/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1715 - rmse: 0.4057 - acc: 0.8186 - val_loss: 0.8503 - val_rmse: 1.6777 - val_acc: 0.3557

Epoch 00097: val_loss did not improve from 0.83156

Epoch 00097: acc improved from 0.81690 to 0.81862, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 98/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1727 - rmse: 0.4131 - acc: 0.8127 - val_loss: 0.8351 - val_rmse: 1.6389 - val_acc: 0.3459

Epoch 00098: val_loss did not improve from 0.83156

Epoch 00098: acc did not improve from 0.81862
Epoch 99/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1710 - rmse:

16330/16330 [==============================] - 48s 3ms/step - loss: 0.1489 - rmse: 0.3597 - acc: 0.8310 - val_loss: 0.8471 - val_rmse: 1.6853 - val_acc: 0.3395

Epoch 00123: val_loss did not improve from 0.81694

Epoch 00123: acc did not improve from 0.83203
Epoch 124/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1469 - rmse: 0.3532 - acc: 0.8344 - val_loss: 0.8278 - val_rmse: 1.6359 - val_acc: 0.3464

Epoch 00124: val_loss did not improve from 0.81694

Epoch 00124: acc improved from 0.83203 to 0.83442, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 125/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1476 - rmse: 0.3648 - acc: 0.8317 - val_loss: 0.8256 - val_rmse: 1.6270 - val_acc: 0.3561

Epoch 00125: val_loss did not improve from 0.81694

Epoch 00125: acc did not improve from 0.83442
Epoch 126/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1462 - rmse: 0.3599 - 


Epoch 00178: val_loss did not improve from 0.81122

Epoch 00178: acc improved from 0.85867 to 0.85915, saving model to D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5
Epoch 179/200
16330/16330 [==============================] - 47s 3ms/step - loss: 0.1162 - rmse: 0.2898 - acc: 0.8566 - val_loss: 0.8238 - val_rmse: 1.6326 - val_acc: 0.3637

Epoch 00179: val_loss did not improve from 0.81122

Epoch 00179: acc did not improve from 0.85915
Epoch 180/200
16330/16330 [==============================] - 49s 3ms/step - loss: 0.1169 - rmse: 0.2907 - acc: 0.8567 - val_loss: 0.8333 - val_rmse: 1.6580 - val_acc: 0.3564

Epoch 00180: val_loss did not improve from 0.81122

Epoch 00180: acc did not improve from 0.85915
Epoch 181/200
16330/16330 [==============================] - 50s 3ms/step - loss: 0.1169 - rmse: 0.2935 - acc: 0.8554 - val_loss: 0.8287 - val_rmse: 1.6432 - val_acc: 0.3640

Epoch 00181: val_loss did not improve from 0.81122

Epoch 00181: acc did not improve from 0

# Inferência (1)

In [37]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/1/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/1/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/1/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
    
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/1/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/1/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/1/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:
Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Inferência (2)

In [39]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/2/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/2/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
    
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/2/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/2/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/2/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:
Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Testes com o valore de Beta

## Mudar a métrica para compreender os valores de perdas

In [27]:
def translation_loss(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    # mean_rot = mean[:, 0:3] * 150
    #mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean_trasl, axis=-1))
    return sqrt

def rotation_loss(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    #mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3]
    #mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean_rot, axis=-1))
    return sqrt

def mean_loss(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3]
    #mean = K.concatenate([mean_rot, mean_trasl])
    sqrt_trasl = K.sqrt(K.mean(mean_rot, axis=-1))
    sqrt_rot = K.sqrt(K.mean(mean_trasl, axis=-1))
    return (sqrt_rot/sqrt_trasl)

def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]* 0.3
    mean_rot = mean[:, 0:3] * 150
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

def vo_loss_mod2(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]* 0.5
    mean_rot = mean[:, 0:3] * 120
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

learning_rate = 0.0005
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod], optimizer=opt, metrics=[rmse, mean_loss, 'acc'])

### Preparando os checkpoints

In [28]:
model_path = "D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthSiameseVO/test-learning-rate-higher/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/
Train on 16330 samples, validate on 6860 samples
Epoch 1/200
16330/16330 [==============================] - 49s 3ms/step - loss: 1.2594 - rmse: 3.1446 - mean_loss: 76.9689 - acc: 0.2515 - val_loss: 0.9317 - val_rmse: 1.5439 - val_mean_loss: 82.9117 - val_acc: 0.2958

Epoch 00001: val_loss improved from inf to 0.93173, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.val.loss.hdf5

Epoch 00001: acc improved from -inf to 0.25150, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 2/200
16330/16330 [==============================] - 48s 3ms/step - loss: 1.0327 - rmse: 3.1379 - mean_loss: 132.9902 - acc: 0.2918 - val_loss: 0.8293 - val_rmse: 1.5433 - val_mean_loss: 172.6612 - val_acc: 0.3069

Epoch 00002: val_loss improved from 0.93173 to 0.82930, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-high

Epoch 19/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.7586 - rmse: 2.1125 - mean_loss: 165.0810 - acc: 0.4604 - val_loss: 0.8441 - val_rmse: 1.6450 - val_mean_loss: 204.2865 - val_acc: 0.3127

Epoch 00019: val_loss did not improve from 0.80720

Epoch 00019: acc improved from 0.45040 to 0.46038, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 20/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.7373 - rmse: 2.0297 - mean_loss: 149.1246 - acc: 0.4776 - val_loss: 0.9015 - val_rmse: 1.6624 - val_mean_loss: 146.4281 - val_acc: 0.3302

Epoch 00020: val_loss did not improve from 0.80720

Epoch 00020: acc improved from 0.46038 to 0.47765, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 21/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.7142 - rmse: 1.9498 - mean_loss: 143.1228 - acc: 0.

Epoch 39/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.4807 - rmse: 1.1401 - mean_loss: 95.5066 - acc: 0.6032 - val_loss: 0.8858 - val_rmse: 1.6793 - val_mean_loss: 128.6345 - val_acc: 0.3515

Epoch 00039: val_loss did not improve from 0.80720

Epoch 00039: acc improved from 0.59786 to 0.60318, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 40/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.4715 - rmse: 1.1279 - mean_loss: 97.2412 - acc: 0.6109 - val_loss: 0.9162 - val_rmse: 1.8048 - val_mean_loss: 183.4147 - val_acc: 0.3532

Epoch 00040: val_loss did not improve from 0.80720

Epoch 00040: acc improved from 0.60318 to 0.61090, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 41/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.4568 - rmse: 1.0783 - mean_loss: 96.0100 - acc: 0.619

Epoch 59/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.3715 - rmse: 0.8281 - mean_loss: 75.8941 - acc: 0.6731 - val_loss: 0.9432 - val_rmse: 1.8281 - val_mean_loss: 163.8126 - val_acc: 0.3391

Epoch 00059: val_loss did not improve from 0.80720

Epoch 00059: acc did not improve from 0.68096
Epoch 60/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.3552 - rmse: 0.7908 - mean_loss: 76.5075 - acc: 0.6805 - val_loss: 0.8616 - val_rmse: 1.6595 - val_mean_loss: 169.8625 - val_acc: 0.3440

Epoch 00060: val_loss did not improve from 0.80720

Epoch 00060: acc did not improve from 0.68096
Epoch 61/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.3612 - rmse: 0.8019 - mean_loss: 74.4389 - acc: 0.6873 - val_loss: 0.8686 - val_rmse: 1.6920 - val_mean_loss: 195.3062 - val_acc: 0.3446

Epoch 00061: val_loss did not improve from 0.80720

Epoch 00061: acc improved from 0.68096 to 0.68726, saving model to D:/KITTI/Graph/DepthSi

16330/16330 [==============================] - 48s 3ms/step - loss: 0.3188 - rmse: 0.6895 - mean_loss: 66.7912 - acc: 0.7114 - val_loss: 0.8849 - val_rmse: 1.7040 - val_mean_loss: 156.9220 - val_acc: 0.3605

Epoch 00082: val_loss did not improve from 0.80720

Epoch 00082: acc did not improve from 0.71304
Epoch 83/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.3106 - rmse: 0.6702 - mean_loss: 67.2017 - acc: 0.7151 - val_loss: 0.8760 - val_rmse: 1.6915 - val_mean_loss: 156.2214 - val_acc: 0.3464

Epoch 00083: val_loss did not improve from 0.80720

Epoch 00083: acc improved from 0.71304 to 0.71513, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 84/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.3113 - rmse: 0.6704 - mean_loss: 63.2259 - acc: 0.7183 - val_loss: 0.8663 - val_rmse: 1.6560 - val_mean_loss: 146.4432 - val_acc: 0.3155

Epoch 00084: val_loss did not improve fro


Epoch 00104: val_loss did not improve from 0.80720

Epoch 00104: acc did not improve from 0.72841
Epoch 105/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2894 - rmse: 0.6027 - mean_loss: 60.0384 - acc: 0.7308 - val_loss: 0.8658 - val_rmse: 1.7002 - val_mean_loss: 197.6582 - val_acc: 0.3525

Epoch 00105: val_loss did not improve from 0.80720

Epoch 00105: acc improved from 0.72841 to 0.73080, saving model to D:/KITTI/Graph/DepthSiameseVO/model/test-learning-rate-higher/weights.best.by.accuracy.hdf5
Epoch 106/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2888 - rmse: 0.6160 - mean_loss: 63.8565 - acc: 0.7305 - val_loss: 0.8644 - val_rmse: 1.6096 - val_mean_loss: 126.3219 - val_acc: 0.3625

Epoch 00106: val_loss did not improve from 0.80720

Epoch 00106: acc did not improve from 0.73080
Epoch 107/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2837 - rmse: 0.5924 - mean_loss: 61.0894 - acc: 0.7309 - val_los

16330/16330 [==============================] - 48s 3ms/step - loss: 0.2759 - rmse: 0.5709 - mean_loss: 57.6903 - acc: 0.7353 - val_loss: 0.8473 - val_rmse: 1.6381 - val_mean_loss: 192.9648 - val_acc: 0.3472

Epoch 00129: val_loss did not improve from 0.80720

Epoch 00129: acc did not improve from 0.74140
Epoch 130/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2835 - rmse: 0.5802 - mean_loss: 57.5284 - acc: 0.7342 - val_loss: 0.8518 - val_rmse: 1.6132 - val_mean_loss: 108.0586 - val_acc: 0.3198

Epoch 00130: val_loss did not improve from 0.80720

Epoch 00130: acc did not improve from 0.74140
Epoch 131/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2672 - rmse: 0.5638 - mean_loss: 60.3114 - acc: 0.7356 - val_loss: 0.8801 - val_rmse: 1.6950 - val_mean_loss: 159.6144 - val_acc: 0.3402

Epoch 00131: val_loss did not improve from 0.80720

Epoch 00131: acc did not improve from 0.74140
Epoch 132/200
16330/16330 [===========================


Epoch 00153: val_loss did not improve from 0.80720

Epoch 00153: acc did not improve from 0.74917
Epoch 154/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2588 - rmse: 0.5365 - mean_loss: 59.2615 - acc: 0.7475 - val_loss: 0.8606 - val_rmse: 1.6793 - val_mean_loss: 187.0230 - val_acc: 0.3372

Epoch 00154: val_loss did not improve from 0.80720

Epoch 00154: acc did not improve from 0.74917
Epoch 155/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2678 - rmse: 0.5530 - mean_loss: 55.9695 - acc: 0.7487 - val_loss: 0.8433 - val_rmse: 1.6522 - val_mean_loss: 214.6897 - val_acc: 0.3555

Epoch 00155: val_loss did not improve from 0.80720

Epoch 00155: acc did not improve from 0.74917
Epoch 156/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2682 - rmse: 0.5490 - mean_loss: 55.7880 - acc: 0.7464 - val_loss: 0.8475 - val_rmse: 1.6174 - val_mean_loss: 144.7049 - val_acc: 0.3738

Epoch 00156: val_loss did not improve f

16330/16330 [==============================] - 48s 3ms/step - loss: 0.2509 - rmse: 0.5156 - mean_loss: 56.8526 - acc: 0.7538 - val_loss: 0.8568 - val_rmse: 1.6820 - val_mean_loss: 257.2732 - val_acc: 0.3700

Epoch 00178: val_loss did not improve from 0.80720

Epoch 00178: acc did not improve from 0.75762
Epoch 179/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2537 - rmse: 0.5189 - mean_loss: 54.3201 - acc: 0.7554 - val_loss: 0.8697 - val_rmse: 1.6003 - val_mean_loss: 114.6400 - val_acc: 0.3399

Epoch 00179: val_loss did not improve from 0.80720

Epoch 00179: acc did not improve from 0.75762
Epoch 180/200
16330/16330 [==============================] - 48s 3ms/step - loss: 0.2615 - rmse: 0.5352 - mean_loss: 57.1682 - acc: 0.7468 - val_loss: 0.8486 - val_rmse: 1.6651 - val_mean_loss: 222.5780 - val_acc: 0.3443

Epoch 00180: val_loss did not improve from 0.80720

Epoch 00180: acc did not improve from 0.75762
Epoch 181/200
16330/16330 [===========================

# Mil épocas

In [ ]:
def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]* 0.3
    mean_rot = mean[:, 0:3] * 150
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

num_epochs = 1000
learning_rate = 0.0001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod], optimizer=opt, metrics=[rmse, 'acc'])

### Preparando os checkpoints

In [ ]:
model_path = "D:/KITTI/Graph/DepthSiameseVO/model/same-1000-epochs/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthSiameseVO/same-1000-epochs/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthSiameseVO/model/same-1000-epochs/
Train on 16330 samples, validate on 6860 samples
Epoch 1/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1503 - rmse: 0.3580 - acc: 0.8139 - val_loss: 0.7986 - val_rmse: 1.5693 - val_acc: 0.3617

Epoch 00001: val_loss improved from inf to 0.79862, saving model to D:/KITTI/Graph/DepthSiameseVO/model/same-1000-epochs/weights.best.by.val.loss.hdf5

Epoch 00001: acc improved from -inf to 0.81390, saving model to D:/KITTI/Graph/DepthSiameseVO/model/same-1000-epochs/weights.best.by.accuracy.hdf5
Epoch 2/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.1185 - rmse: 0.2632 - acc: 0.8331 - val_loss: 0.8058 - val_rmse: 1.5966 - val_acc: 0.3496

Epoch 00002: val_loss did not improve from 0.79862

Epoch 00002: acc improved from 0.81390 to 0.83313, saving model to D:/KITTI/Graph/DepthSiameseVO/model/same-1000-epochs/weights.best.by.accuracy.hdf5
Epoch 3/1000
16330/16330 [======================

Epoch 23/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0602 - rmse: 0.1261 - acc: 0.8936 - val_loss: 0.8111 - val_rmse: 1.6075 - val_acc: 0.3608

Epoch 00023: val_loss did not improve from 0.79738

Epoch 00023: acc did not improve from 0.89645
Epoch 24/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0586 - rmse: 0.1227 - acc: 0.8963 - val_loss: 0.8055 - val_rmse: 1.5909 - val_acc: 0.3493

Epoch 00024: val_loss did not improve from 0.79738

Epoch 00024: acc did not improve from 0.89645
Epoch 25/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0584 - rmse: 0.1223 - acc: 0.8961 - val_loss: 0.8046 - val_rmse: 1.5909 - val_acc: 0.3558

Epoch 00025: val_loss did not improve from 0.79738

Epoch 00025: acc did not improve from 0.89645
Epoch 26/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0578 - rmse: 0.1184 - acc: 0.8996 - val_loss: 0.8044 - val_rmse: 1.5888 - val_acc: 0.3593

Epoch 

Epoch 50/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0502 - rmse: 0.1074 - acc: 0.9107 - val_loss: 0.8060 - val_rmse: 1.5945 - val_acc: 0.3603

Epoch 00050: val_loss did not improve from 0.79738

Epoch 00050: acc did not improve from 0.91298
Epoch 51/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0509 - rmse: 0.1102 - acc: 0.9122 - val_loss: 0.8110 - val_rmse: 1.6089 - val_acc: 0.3640

Epoch 00051: val_loss did not improve from 0.79738

Epoch 00051: acc did not improve from 0.91298
Epoch 52/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0515 - rmse: 0.1102 - acc: 0.9115 - val_loss: 0.8137 - val_rmse: 1.6194 - val_acc: 0.3599

Epoch 00052: val_loss did not improve from 0.79738

Epoch 00052: acc did not improve from 0.91298
Epoch 53/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0512 - rmse: 0.1104 - acc: 0.9091 - val_loss: 0.8131 - val_rmse: 1.6140 - val_acc: 0.3614

Epoch 


Epoch 00106: val_loss did not improve from 0.79738

Epoch 00106: acc did not improve from 0.92198
Epoch 107/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0458 - rmse: 0.0994 - acc: 0.9206 - val_loss: 0.8171 - val_rmse: 1.6267 - val_acc: 0.3519

Epoch 00107: val_loss did not improve from 0.79738

Epoch 00107: acc did not improve from 0.92198
Epoch 108/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0456 - rmse: 0.1003 - acc: 0.9225 - val_loss: 0.8156 - val_rmse: 1.6229 - val_acc: 0.3656

Epoch 00108: val_loss did not improve from 0.79738

Epoch 00108: acc improved from 0.92198 to 0.92254, saving model to D:/KITTI/Graph/DepthSiameseVO/model/same-1000-epochs/weights.best.by.accuracy.hdf5
Epoch 109/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0453 - rmse: 0.0983 - acc: 0.9190 - val_loss: 0.8134 - val_rmse: 1.6173 - val_acc: 0.3590

Epoch 00109: val_loss did not improve from 0.79738

Epoch 00109: acc did 

16330/16330 [==============================] - 48s 3ms/step - loss: 0.0439 - rmse: 0.0975 - acc: 0.9219 - val_loss: 0.8201 - val_rmse: 1.6337 - val_acc: 0.3593

Epoch 00136: val_loss did not improve from 0.79738

Epoch 00136: acc did not improve from 0.92456
Epoch 137/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0449 - rmse: 0.0995 - acc: 0.9222 - val_loss: 0.8167 - val_rmse: 1.6246 - val_acc: 0.3550

Epoch 00137: val_loss did not improve from 0.79738

Epoch 00137: acc did not improve from 0.92456
Epoch 138/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0445 - rmse: 0.0976 - acc: 0.9239 - val_loss: 0.8177 - val_rmse: 1.6278 - val_acc: 0.3583

Epoch 00138: val_loss did not improve from 0.79738

Epoch 00138: acc did not improve from 0.92456
Epoch 139/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0434 - rmse: 0.0961 - acc: 0.9223 - val_loss: 0.8229 - val_rmse: 1.6395 - val_acc: 0.3624

Epoch 00139: val_

...


Epoch 00281: val_loss did not improve from 0.79738

Epoch 00281: acc did not improve from 0.93111
Epoch 282/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0407 - rmse: 0.0913 - acc: 0.9297 - val_loss: 0.8242 - val_rmse: 1.6451 - val_acc: 0.3595

Epoch 00282: val_loss did not improve from 0.79738

Epoch 00282: acc did not improve from 0.93111
Epoch 283/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0406 - rmse: 0.0906 - acc: 0.9309 - val_loss: 0.8231 - val_rmse: 1.6403 - val_acc: 0.3627

Epoch 00283: val_loss did not improve from 0.79738

Epoch 00283: acc did not improve from 0.93111
Epoch 284/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0408 - rmse: 0.0919 - acc: 0.9293 - val_loss: 0.8273 - val_rmse: 1.6539 - val_acc: 0.3634

Epoch 00284: val_loss did not improve from 0.79738

Epoch 00284: acc did not improve from 0.93111
Epoch 285/1000
16330/16330 [==============================] - 48s 3ms/step - lo


Epoch 00340: val_loss did not improve from 0.79738

Epoch 00340: acc did not improve from 0.93288
Epoch 341/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0396 - rmse: 0.0891 - acc: 0.9328 - val_loss: 0.8240 - val_rmse: 1.6456 - val_acc: 0.3577

Epoch 00341: val_loss did not improve from 0.79738

Epoch 00341: acc did not improve from 0.93288
Epoch 342/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0397 - rmse: 0.0896 - acc: 0.9295 - val_loss: 0.8289 - val_rmse: 1.6576 - val_acc: 0.3558

Epoch 00342: val_loss did not improve from 0.79738

Epoch 00342: acc did not improve from 0.93288
Epoch 343/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0393 - rmse: 0.0877 - acc: 0.9279 - val_loss: 0.8262 - val_rmse: 1.6535 - val_acc: 0.3630

Epoch 00343: val_loss did not improve from 0.79738

Epoch 00343: acc did not improve from 0.93288
Epoch 344/1000
16330/16330 [==============================] - 48s 3ms/step - lo


Epoch 00399: val_loss did not improve from 0.79738

Epoch 00399: acc did not improve from 0.93368
Epoch 400/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0394 - rmse: 0.0891 - acc: 0.9299 - val_loss: 0.8286 - val_rmse: 1.6605 - val_acc: 0.3620

Epoch 00400: val_loss did not improve from 0.79738

Epoch 00400: acc did not improve from 0.93368
Epoch 401/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0396 - rmse: 0.0905 - acc: 0.9287 - val_loss: 0.8271 - val_rmse: 1.6542 - val_acc: 0.3574

Epoch 00401: val_loss did not improve from 0.79738

Epoch 00401: acc did not improve from 0.93368
Epoch 402/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0395 - rmse: 0.0897 - acc: 0.9292 - val_loss: 0.8291 - val_rmse: 1.6599 - val_acc: 0.3571

Epoch 00402: val_loss did not improve from 0.79738

Epoch 00402: acc did not improve from 0.93368
Epoch 403/1000
16330/16330 [==============================] - 48s 3ms/step - lo


Epoch 00458: val_loss did not improve from 0.79738

Epoch 00458: acc did not improve from 0.93515
Epoch 459/1000
16330/16330 [==============================] - 50s 3ms/step - loss: 0.0390 - rmse: 0.0885 - acc: 0.9328 - val_loss: 0.8303 - val_rmse: 1.6599 - val_acc: 0.3622

Epoch 00459: val_loss did not improve from 0.79738

Epoch 00459: acc did not improve from 0.93515
Epoch 460/1000
16330/16330 [==============================] - 50s 3ms/step - loss: 0.0388 - rmse: 0.0868 - acc: 0.9301 - val_loss: 0.8297 - val_rmse: 1.6593 - val_acc: 0.3668

Epoch 00460: val_loss did not improve from 0.79738

Epoch 00460: acc did not improve from 0.93515
Epoch 461/1000
16330/16330 [==============================] - 50s 3ms/step - loss: 0.0380 - rmse: 0.0856 - acc: 0.9313 - val_loss: 0.8291 - val_rmse: 1.6580 - val_acc: 0.3666

Epoch 00461: val_loss did not improve from 0.79738

Epoch 00461: acc did not improve from 0.93515
Epoch 462/1000
16330/16330 [==============================] - 50s 3ms/step - lo

16330/16330 [==============================] - 48s 3ms/step - loss: 0.0385 - rmse: 0.0869 - acc: 0.9326 - val_loss: 0.8356 - val_rmse: 1.6749 - val_acc: 0.3583

Epoch 00488: val_loss did not improve from 0.79738

Epoch 00488: acc did not improve from 0.93558
Epoch 489/1000
16330/16330 [==============================] - 49s 3ms/step - loss: 0.0384 - rmse: 0.0862 - acc: 0.9346 - val_loss: 0.8293 - val_rmse: 1.6575 - val_acc: 0.3656

Epoch 00489: val_loss did not improve from 0.79738

Epoch 00489: acc did not improve from 0.93558
Epoch 490/1000
16330/16330 [==============================] - 49s 3ms/step - loss: 0.0380 - rmse: 0.0847 - acc: 0.9307 - val_loss: 0.8326 - val_rmse: 1.6692 - val_acc: 0.3647

Epoch 00490: val_loss did not improve from 0.79738

Epoch 00490: acc did not improve from 0.93558
Epoch 491/1000
16330/16330 [==============================] - 48s 3ms/step - loss: 0.0384 - rmse: 0.0868 - acc: 0.9328 - val_loss: 0.8307 - val_rmse: 1.6651 - val_acc: 0.3622

Epoch 00491: val_

# Usando quaternion

## Training pose

### Transformation

In [46]:
from pyquaternion import Quaternion

T_train_quaternion = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_training_0[name], label_training_1[name]):
        transformation = pose_0.dot(np.linalg.inv(pose_1))
        t_aux = transformation[0:3, 3]        
        t = np.zeros((1,3))
        
        for i in range(0,3):
            t[0, i] = t_aux[i]
            cont += 1
            
        quaternion = np.array(Quaternion(matrix=transformation))
        print(quaternion)
        T_aux.append(np.concatenate((quaternion, t),1))
    
        T_train_quaternion[name] = T_aux    

1.000 -0.001i +0.001j +0.000k


ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
training_poses_transformed_quaternion = []

for sequence in kitti_train_dirs:
  
    for pose in T_train_quaternion[sequence]:
        training_poses_transformed_quaternion.append(pose)

print("Training > size right: %s, size left: %s, size label: %s"%
      (len(training_r), len(training_l), len(training_poses)))

### Test pose

### Transformation

In [ ]:
T_test_quaternion = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_test_0[name], label_test_1[name]):
        transformation = pose_0.dot(np.linalg.inv(pose_1))
        t_aux = transformation[0:3, 3]        
        t = np.zeros((1,3))
        
        for i in range(0,3):
            t[0, i] = t_aux[i]
            cont += 1
            
        quaternion = np.array(Quaternion(matrix=transformation))
        
        T_aux.append(np.concatenate((quaternion, t),1))
    
    T_test_quaternion[name] = T_aux

In [ ]:
test_poses_transformed_quaternion = []

for sequence in kitti_test_dirs:    
        for pose in T_test_quaternion[sequence]:
            test_poses_transformed_quaternion.append(pose)
    
print("Test > Size right: %s, size left: %s, size label: %s"%
      (len(test_r), len(test_l), len(test_poses)))

### Preparando a entrada para o padrão Tensorflow

In [ ]:
y_training = np.reshape(training_poses_transformed_quaternion, (len(training_poses), 6))
y_test = np.reshape(test_poses_transformed_quaternion, (len(test_poses), 6))
print(np.shape(y_training))

### Definindo a arquitetura

In [41]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

learning_rate = 0.0001
epochs = 100

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(LeakyReLU(alpha=0.6))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(LeakyReLU(alpha=0.6))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(LeakyReLU(alpha=0.6))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(LeakyReLU(alpha=0.6))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(2048, kernel_initializer='normal')(abs_diff)
activation1 = LeakyReLU(alpha=0.6)(dense_layer)
#dense_layer1 = Dense(2048, kernel_initializer='normal')(activation1)
#activation2 = LeakyReLU(alpha=0.5)(dense_layer1)
dense_layer2 = Dense(2048, kernel_initializer='normal')(activation1)
activation3 = LeakyReLU(alpha=0.2)(dense_layer2)
output = Dense(7, name='predictions', kernel_initializer='normal')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 8704)         92672       input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 8704)         0           sequential_5[1][0]               
          

In [42]:
def vo_loss2(y_true, y_pred):
    loss_value = vo_loss(y_true, y_pred)*100    
    return loss_value

learning_rate = 0.0001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss], optimizer=opt, metrics=[rmse, 'acc'])

In [43]:
num_epochs = 200

### Preparando os checkpoints

In [44]:
model_path = "D:/KITTI/Graph/DepthSiameseVO/model/3/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthSiameseVO/3/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthSiameseVO/model/3/
Train on 16330 samples, validate on 6860 samples
Epoch 1/200


InvalidArgumentError: Incompatible shapes: [16,6] vs. [16,7]
	 [[Node: metrics_9/rmse/sub = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_predictions_target_9_0_3/_1799, predictions_4/BiasAdd)]]
	 [[Node: metrics_9/rmse/Mean_1/_1949 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1308_metrics_9/rmse/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'metrics_9/rmse/sub', defined at:
  File "C:\Users\vinic\AppData\Local\conda\conda\envs\tf2\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\vinic\AppData\Local\conda\conda\envs\tf2\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\vinic\AppData\Local\conda\conda\envs\tf2\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
   

## Mudar a função de perda

In [ ]:
def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]* 0.3
    mean_rot = mean[:, 0:3] * 150
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

learning_rate = 0.0001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod], optimizer=opt, metrics=[rmse, 'acc'])



### Preparando os checkpoints

In [ ]:
model_path = "D:/KITTI/Graph/DepthSiameseVO/model/4/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthSiameseVO/4/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

# Inferência (3)

In [ ]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/3/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/3/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/3/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
    
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/3/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/3/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/3/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

# Inferência (4)

In [ ]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/4/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/4/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/4/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
    
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthSiameseVO/model/4/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/4/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = Quaternion(element[0:4]).rotation_matrix
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/DepthSiameseVO/Results/4/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

# Alternando arquiteturas e dados

# Definindo a arquitetura
***

### Usando Relu no lugar de Leaky Relu

* Dados originais

In [80]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(1028)(abs_diff)
activation1 = Activation('relu')(dense_layer)
dense_layer1 = Dense(1028)(activation1)
activation2 = Activation('relu')(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = Activation('relu')(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(3, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 1028)         9041412     input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 1028)         0           sequential_6[1][0]               
          

## Dados transformados

### Preparando os checkpoints e treinando

In [82]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(1028)(abs_diff)
activation1 = Activation('relu')(dense_layer)
dense_layer1 = Dense(1028)(activation1)
activation2 = Activation('relu')(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = Activation('relu')(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(3, name='predictions')(activation3)

model = Model([input_a, input_b], output)

def vo_loss2(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3] * 50
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt*1000

# learning_rate = 0.00001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss2], optimizer=opt, metrics=[rmse, 'acc'])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_7 (Sequential)       (None, 1028)         9041412     input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, 1028)         0           sequential_7[1][0]               
          

### Preparando a entrada para o padrão Tensorflow

In [42]:
flag_ypr_t = False
flag_ypr_t_transformed = True
flag_only_rotation = False
flag_only_translation = False


if(flag_ypr_t):

    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses, (len(training_poses), 6))
    y_test = np.reshape(test_poses, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_ypr_t_transformed):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_transformed, (len(training_poses), 6))
    y_test = np.reshape(test_poses_transformed, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_only_rotation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_rotation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_rotation, (len(test_poses), 3))
    print(np.shape(y_training))
    
elif(flag_only_translation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_translation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_translation, (len(test_poses), 3))
    print(np.shape(y_training))

(16330, 47, 155, 1)
(6860, 47, 155, 1)
(16330, 6)


### Preparando os checkpoints

In [43]:
model_path = "D:/KITTI/Graph/DepthVO_RELU_transformed/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_RELU_transformed', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=50, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/50
16330/16330 [==============================] - 33s 2ms/step - loss: 1770.8765 - rmse: 3.1792 - acc: 0.2202 - val_loss: 1464.1236 - val_rmse: 1.5331 - val_acc: 0.2257

Epoch 00001: val_loss improved from inf to 1464.12359, saving model to D:/KITTI/Graph/DepthVO_RELU_transformed/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.22021, saving model to D:/KITTI/Graph/DepthVO_RELU_transformed/model/weights.best.by.accuracy.hdf5
Epoch 2/50
16330/16330 [==============================] - 32s 2ms/step - loss: 1750.1323 - rmse: 3.1444 - acc: 0.2358 - val_loss: 1466.6845 - val_rmse: 1.5357 - val_acc: 0.1776

Epoch 00002: val_loss did not improve from 1464.12359

Epoch 00002: acc improved from 0.22021 to 0.23576, saving model to D:/KITTI/Graph/DepthVO_RELU_transformed/model/weights.best.by.accuracy.hdf5
Epoch 3/50
16330/16330 [==============================] - 33s 2ms/step - loss: 1683.2949 - rmse: 3.0106 - acc: 0.3831 - v

## Inferência

In [83]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_RELU_transformed/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/output_to_draw/fifth/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/output_to_draw/fifth/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:

## Dados transformados

### Preparando os checkpoints e treinando

In [88]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
#activation2 = Activation('relu')(dense_layer1)
dense_layer2 = Dense(64)(abs_diff)
activation3 = Activation('relu')(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(3, name='predictions')(activation3)

model = Model([input_a, input_b], output)

def vo_loss2(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3] * 50
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt*1000

# learning_rate = 0.00001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss2], optimizer=opt, metrics=[rmse, 'acc'])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_10 (Sequential)      (None, 1028)         9041412     input_19[0][0]                   
                                                                 input_20[0][0]                   
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 1028)         0           sequential_10[1][0]              
          

### Preparando a entrada para o padrão Tensorflow

In [54]:
flag_ypr_t = False
flag_ypr_t_transformed = True
flag_only_rotation = False
flag_only_translation = False


if(flag_ypr_t):

    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses, (len(training_poses), 6))
    y_test = np.reshape(test_poses, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_ypr_t_transformed):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_transformed, (len(training_poses), 6))
    y_test = np.reshape(test_poses_transformed, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_only_rotation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_rotation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_rotation, (len(test_poses), 3))
    print(np.shape(y_training))
    
elif(flag_only_translation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_translation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_translation, (len(test_poses), 3))
    print(np.shape(y_training))

(16330, 47, 155, 1)
(6860, 47, 155, 1)
(16330, 6)


### Preparando os checkpoints

In [55]:
model_path = "D:/KITTI/Graph/DepthVO_Fully_Connected_transformed/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_Fully_Connected_transformed', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=50, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVO_Fully_Connected_transformed/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/50
16330/16330 [==============================] - 31s 2ms/step - loss: 1787.1150 - rmse: 3.1751 - acc: 0.2375 - val_loss: 1454.4293 - val_rmse: 1.5252 - val_acc: 0.2362

Epoch 00001: val_loss improved from inf to 1454.42932, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_transformed/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.23754, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_transformed/model/weights.best.by.accuracy.hdf5
Epoch 2/50
16330/16330 [==============================] - 30s 2ms/step - loss: 1763.3797 - rmse: 3.1683 - acc: 0.2566 - val_loss: 1454.7221 - val_rmse: 1.5256 - val_acc: 0.2376

Epoch 00002: val_loss did not improve from 1454.42932

Epoch 00002: acc improved from 0.23754 to 0.25664, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_transformed/model/weights.best.by.accuracy.hdf5
Epoch 3/50
16330/16330 [======

## Inferência

In [89]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_Fully_Connected_transformed/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/output_to_draw/eighth/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/output_to_draw/eighth/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Definindo a arquitetura
***

## Usando concatenate no lugar de get_abs_diff

* Dados originais

### Dados transformados

### Preparando os checkpoints e treinando

In [103]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
dense_layer = Dense(1028)(concatenetion_layer)
activation1 = Activation('relu')(dense_layer)
dense_layer1 = Dense(1028)(activation1)
activation2 = Activation('relu')(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = Activation('relu')(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(3, name='predictions')(activation3)

model = Model([input_a, input_b], output)

def vo_loss2(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3] * 50
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt*1000

# learning_rate = 0.00001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss2], optimizer=opt, metrics=[rmse, 'acc'])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_18 (Sequential)      (None, 1028)         9041412     input_35[0][0]                   
                                                                 input_36[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 2056)         0           sequential_18[1][0]              
          

### Preparando a entrada para o padrão Tensorflow

In [67]:
flag_ypr_t = False
flag_ypr_t_transformed = True
flag_only_rotation = False
flag_only_translation = False


if(flag_ypr_t):

    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses, (len(training_poses), 6))
    y_test = np.reshape(test_poses, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_ypr_t_transformed):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_transformed, (len(training_poses), 6))
    y_test = np.reshape(test_poses_transformed, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_only_rotation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_rotation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_rotation, (len(test_poses), 3))
    print(np.shape(y_training))
    
elif(flag_only_translation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_translation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_translation, (len(test_poses), 3))
    print(np.shape(y_training))

(16330, 47, 155, 1)
(6860, 47, 155, 1)
(16330, 6)


### Preparando os checkpoints

In [68]:
model_path = "D:/KITTI/Graph/DepthVO_concatenate_transformed/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_concatenate_transformed', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVO_concatenate_transformed/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 34s 2ms/step - loss: 1788.8493 - rmse: 3.1523 - acc: 0.2100 - val_loss: 1454.0441 - val_rmse: 1.5251 - val_acc: 0.2401

Epoch 00001: val_loss improved from inf to 1454.04411, saving model to D:/KITTI/Graph/DepthVO_concatenate_transformed/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.20998, saving model to D:/KITTI/Graph/DepthVO_concatenate_transformed/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 34s 2ms/step - loss: 1763.9305 - rmse: 3.1649 - acc: 0.2592 - val_loss: 1454.3986 - val_rmse: 1.5255 - val_acc: 0.2401

Epoch 00002: val_loss did not improve from 1454.04411

Epoch 00002: acc improved from 0.20998 to 0.25922, saving model to D:/KITTI/Graph/DepthVO_concatenate_transformed/model/weights.best.by.accuracy.hdf5
Epoch 3/25
16330/16330 [======================

## Inferência

In [104]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_concatenate_transformed/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/output_to_draw/eleventh/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/output_to_draw/eleventh/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


## Diminuindo a quantidade de Fully connect layers:

### Dados transformados

#### Preparando os checkpoints e treinando

In [109]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
#activation2 = Activation('relu')(dense_layer1)
dense_layer2 = Dense(64)(concatenetion_layer)
activation3 = Activation('relu')(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(3, name='predictions')(activation3)

model = Model([input_a, input_b], output)

def vo_loss2(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3] * 50
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt*1000

# learning_rate = 0.00001
opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss2], optimizer=opt, metrics=[rmse, 'acc'])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_21 (Sequential)      (None, 1028)         9041412     input_41[0][0]                   
                                                                 input_42[0][0]                   
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 2056)         0           sequential_21[1][0]              
          

### Preparando a entrada para o padrão Tensorflow

In [79]:
flag_ypr_t = False
flag_ypr_t_transformed = True
flag_only_rotation = False
flag_only_translation = False


if(flag_ypr_t):

    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses, (len(training_poses), 6))
    y_test = np.reshape(test_poses, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_ypr_t_transformed):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_transformed, (len(training_poses), 6))
    y_test = np.reshape(test_poses_transformed, (len(test_poses), 6))
    print(np.shape(y_training))
    
elif(flag_only_rotation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_rotation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_rotation, (len(test_poses), 3))
    print(np.shape(y_training))
    
elif(flag_only_translation):
    training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
    training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

    print(np.shape(training_r))

    test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
    test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
    print(np.shape(test_r))

    y_training = np.reshape(training_poses_translation, (len(training_poses), 3))
    y_test = np.reshape(test_poses_translation, (len(test_poses), 3))
    print(np.shape(y_training))

(16330, 47, 155, 1)
(6860, 47, 155, 1)
(16330, 6)


### Preparando os checkpoints

In [80]:
model_path = "D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 31s 2ms/step - loss: 1814.8221 - rmse: 3.1878 - acc: 0.2516 - val_loss: 1454.0738 - val_rmse: 1.5252 - val_acc: 0.2401

Epoch 00001: val_loss improved from inf to 1454.07380, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.25156, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 32s 2ms/step - loss: 1763.2194 - rmse: 3.1514 - acc: 0.2600 - val_loss: 1454.4035 - val_rmse: 1.5255 - val_acc: 0.2401

Epoch 00002: val_loss did not improve from 1454.07380

Epoch 00002: acc improved from 0.25156 to 0.25995, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed/model/weights.best

## Inferência

In [110]:
for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))

    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_transformed/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
        

    
    np.savetxt("D:/KITTI/output_to_draw/fourteenth/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    aux_list = []
    aux_list.append(init_mat)
    
    for element in prediction:
        matrix = rb.rpy2tr(element[0:3])
                
        for i in range(3):
            translation = element[3:6]
            matrix[0, 3] = translation[0]
            matrix[1, 3] = translation[1]
            matrix[2, 3] = translation[2]
            
        # output = matrix[0:3,:].flatten()
        #print(np.shape(output))
        # sequence_GT[cont, :] = output
        aux_list.append(matrix)
        
        
    prediction_aux = aux_list
    pose_ant = np.identity(4)
    
    cont = 1
    
    for transformation in prediction_aux[1:]:
        inv_transformation = pose_ant.dot(transformation)
        output = inv_transformation[0:3,:].flatten()
        sequence_GT[cont, :] = output
        cont += 1
        pose_ant = inv_transformation
    
    np.savetxt("D:/KITTI/output_to_draw/fourteenth/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Thesis ground Truth

## Read and transform training pose (of thesis)

In [18]:
cont = 0
label_training_thesis_0 = {}
label_training_thesis_1 = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   

        label_aux.append(matrix)
        
        
    label_training_thesis_0[name] = label_aux[:-1]
    label_training_thesis_1[name] = label_aux[1:]


## Transformation

In [19]:
from math import asin, atan2, sqrt

T_train_thesis = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_training_thesis_0[name], label_training_thesis_1[name]):
        
        Tx = pose_1[0][3]
        Tz = pose_1[2][3]
        Tx_ant = pose_0[0][3]
        Tz_ant = pose_0[2][3]

        Ri_1 = pose_1[0][0]
        Ri_3 = pose_1[0][2]    
        Ri_1_ant = pose_0[0][0]
        Ri_3_ant = pose_0[0][2]
        
        Ti = [Tx, Tz]
        Ti_ant = [Tx_ant, Tz_ant]

        displacement = sqrt(sum([(a - b) ** 2 for a, b in zip(Ti, Ti_ant)]))
        delta_Theta = atan2(Ri_3, Ri_1) - atan2(Ri_3_ant, Ri_1_ant)
    
        T_aux.append([delta_Theta, displacement])
    
    T_train_thesis[name] = T_aux
    print(len(T_train_thesis[name]))
    

4540
1100
4660
800
270
2760
1100
1100


## Read and transform test pose (of thesis)

In [20]:
cont = 0
label_test_thesis_0 = {}
label_test_thesis_1 = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   

        label_aux.append(matrix)
        
        
    label_test_thesis_0[name] = label_aux[:-1]
    label_test_thesis_1[name] = label_aux[1:]


## Transformation

In [21]:
T_test_thesis = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_test_thesis_0[name], label_test_thesis_1[name]):
        
        Tx = pose_1[0][3]
        Tz = pose_1[2][3]
        Tx_last = pose_0[0][3]
        Tz_last = pose_0[2][3]

        Ri_1 = pose_1[0][0]
        Ri_3 = pose_1[0][2]    
        Ri_1_last = pose_0[0][0]
        Ri_3_last = pose_0[0][2]
        
        Ti = [Tx, Tz]
        Ti_last = [Tx_last, Tz_last]

        displacement = sqrt(sum([(a - b) ** 2 for a, b in zip(Ti, Ti_last)]))
        delta_Theta = atan2(Ri_3, Ri_1) - atan2(Ri_3_last, Ri_1_last)
    
        T_aux.append([delta_Theta, displacement])
    
    T_test_thesis[name] = T_aux
    # print(T_train)
    

In [22]:
training_poses_transformed_thesis = []

for sequence in kitti_train_dirs:
  
    for pose in T_train_thesis[sequence]:
        training_poses_transformed_thesis.append(pose)

In [23]:
test_poses_transformed_thesis = []

for sequence in kitti_test_dirs:    
        for pose in T_test_thesis[sequence]:
            test_poses_transformed_thesis.append(pose)
            

### Preparando a entrada para o padrão Tensorflow

In [24]:
y_training = np.reshape(training_poses_transformed_thesis, (len(training_poses_transformed_thesis), 2))
y_test = np.reshape(test_poses_transformed_thesis, (len(test_poses_transformed_thesis), 2))
print(np.shape(y_training))

(16330, 2)


## Treinamentos

# Definindo a arquitetura
***

### Usando Leaky Relu

* Dados originais

In [15]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(1028)(abs_diff)
activation1 = LeakyReLU(alpha=0.5)(dense_layer)
dense_layer1 = Dense(1028)(activation1)
activation2 = LeakyReLU(alpha=0.5)(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = LeakyReLU(alpha=0.2)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1028)         9041412     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1028)         0           sequential_1[1][0]               
          

### Preparando os checkpoints

In [119]:
model_path = "D:/KITTI/Graph/DepthVO_RELU_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_RELU_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 37s 2ms/step - loss: 0.1346 - rmse: 0.3275 - acc: 0.9979 - val_loss: 0.0849 - val_rmse: 0.2197 - val_acc: 0.9991

Epoch 00001: val_loss improved from inf to 0.08495, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99792, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 38s 2ms/step - loss: 0.0656 - rmse: 0.2184 - acc: 0.9989 - val_loss: 0.0734 - val_rmse: 0.1969 - val_acc: 0.9990

Epoch 00002: val_loss improved from 0.08495 to 0.07337, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.99792 to 0.99890, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis/model/weights.best.by.accuracy.hdf5
Epoch 3/25
16330/16330 [==============================] - 37s 2ms/step - l

## Inferência

In [17]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_RELU_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-1/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-1/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:




### Preparando os checkpoints e treinando

In [18]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
#activation2 = Activation('relu')(dense_layer1)

activation2 = LeakyReLU(alpha=0.5)(abs_diff)
dense_layer2 = Dense(64)(activation2)
activation3 = LeakyReLU(alpha=0.2)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 1028)         9041412     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1028)         0           sequential_2[1][0]               
          

### Preparando os checkpoints

In [121]:
model_path = "D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_Fully_Connected_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 32s 2ms/step - loss: 0.1776 - rmse: 0.3841 - acc: 0.9955 - val_loss: 0.0870 - val_rmse: 0.2368 - val_acc: 0.9984

Epoch 00001: val_loss improved from inf to 0.08696, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99553, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 32s 2ms/step - loss: 0.0692 - rmse: 0.2293 - acc: 0.9985 - val_loss: 0.0813 - val_rmse: 0.2209 - val_acc: 0.9988

Epoch 00002: val_loss improved from 0.08696 to 0.08128, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.99553 to 0.99847, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5
Epoch 3/25
16330/16330 [======

## Inferência

In [19]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-2/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-2/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Definindo a arquitetura
***

### Usando concatenate no lugar de get_abs_diff

* Dados originais

### Preparando os checkpoints e treinando

In [20]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
dense_layer = Dense(1028)(concatenetion_layer)
activation1 = LeakyReLU(alpha=0.5)(dense_layer)
dense_layer1 = Dense(1028)(activation1)
activation2 = LeakyReLU(alpha=0.5)(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = LeakyReLU(alpha=0.2)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 1028)         9041412     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 2056)         0           sequential_3[1][0]               
          

### Preparando os checkpoints

In [125]:
model_path = "D:/KITTI/Graph/DepthVO_concatenate_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_concatenate_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 39s 2ms/step - loss: 0.2516 - rmse: 0.2567 - acc: 0.9885 - val_loss: 0.0831 - val_rmse: 0.2074 - val_acc: 0.9980

Epoch 00001: val_loss improved from inf to 0.08315, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.98855, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 39s 2ms/step - loss: 0.0459 - rmse: 0.1678 - acc: 0.9968 - val_loss: 0.0725 - val_rmse: 0.1872 - val_acc: 0.9988

Epoch 00002: val_loss improved from 0.08315 to 0.07247, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.98855 to 0.99675, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.by.accuracy.hdf5
Epoch 3/25
16330/16330 [======================

## Inferência

In [21]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-3/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-3/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:




### Preparando os checkpoints e treinando

In [22]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(32, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    seq.add(Dense(1028))
    seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
activation2 = LeakyReLU(alpha=0.5)(concatenetion_layer)
dense_layer2 = Dense(64)(activation2)
activation3 = LeakyReLU(alpha=0.2)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 1028)         9041412     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 2056)         0           sequential_4[1][0]               
          

### Preparando os checkpoints

In [29]:
model_path = "D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 32s 2ms/step - loss: 0.1909 - rmse: 0.2796 - acc: 0.9873 - val_loss: 0.0820 - val_rmse: 0.2205 - val_acc: 0.9985

Epoch 00001: val_loss improved from inf to 0.08203, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.98732, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 30s 2ms/step - loss: 0.0532 - rmse: 0.1898 - acc: 0.9976 - val_loss: 0.0817 - val_rmse: 0.2075 - val_acc: 0.9990

Epoch 00002: val_loss improved from 0.08203 to 0.08175, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.98732 to 0.99755, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best

## Inferência

In [23]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-4/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-4/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Expanding network size

# Definindo a arquitetura
***

### Usando Leaky Relu

* Dados originais

In [37]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dense_layer2 = Dense(32)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

learning_rate = 0.00001

def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt


opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_12 (Sequential)      (None, 50688)        1918848     input_23[0][0]                   
                                                                 input_24[0][0]                   
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, 50688)        0           sequential_12[1][0]              
          

### Preparando os checkpoints

In [38]:
model_path = "D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 310s 19ms/step - loss: 0.1651 - rmse: 0.3688 - acc: 0.9945 - val_loss: 0.0836 - val_rmse: 0.2249 - val_acc: 0.9895

Epoch 00001: val_loss improved from inf to 0.08360, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99449, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 308s 19ms/step - loss: 0.0669 - rmse: 0.2231 - acc: 0.9944 - val_loss: 0.0697 - val_rmse: 0.1876 - val_acc: 0.9980

Epoch 00002: val_loss improved from 0.08360 to 0.06970, saving model to D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network/model/weights.best.hdf5

Epoch 00002: acc did not improve from 0.99449
Epoch 3/25
16330/16330 [==============================] - 307s 19ms/step - loss: 0.0513 - rmse: 0.1859 - acc: 0

KeyboardInterrupt: 

## Inferência

In [25]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-1-expanded/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-1-expanded/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


## Change loss function

In [39]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dense_layer2 = Dense(32)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

learning_rate = 0.00001

def vo_loss_mod_thesis(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt


opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod_thesis], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_13 (Sequential)      (None, 50688)        1918848     input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 50688)        0           sequential_13[1][0]              
          

### Preparando os checkpoints

In [40]:
model_path = "D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 313s 19ms/step - loss: 0.5510 - rmse: 0.4912 - acc: 0.9963 - val_loss: 0.6227 - val_rmse: 0.3154 - val_acc: 0.9991

Epoch 00001: val_loss improved from inf to 0.62267, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99626, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 313s 19ms/step - loss: 0.5079 - rmse: 0.4222 - acc: 0.9994 - val_loss: 0.6055 - val_rmse: 0.2927 - val_acc: 0.9991

Epoch 00002: val_loss improved from 0.62267 to 0.60548, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/weights.best.hdf5

Epoch 00002: acc improved from 0.99626 to 0.99939, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/weights.best.by.accuracy.hdf5
Epo


Epoch 00025: val_loss did not improve from 0.51288

Epoch 00025: acc did not improve from 0.99945


## Inferência

In [42]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-2-expanded/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-2-expanded/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Treinar mil épocas

## Change loss function

In [43]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dense_layer2 = Dense(32)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

learning_rate = 0.00001

def vo_loss_mod_thesis(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

num_epochs = 500

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod_thesis], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_14 (Sequential)      (None, 50688)        1918848     input_27[0][0]                   
                                                                 input_28[0][0]                   
__________________________________________________________________________________________________
lambda_14 (Lambda)              (None, 50688)        0           sequential_14[1][0]              
          

### Preparando os checkpoints

In [44]:
model_path = "D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/500
16330/16330 [==============================] - 310s 19ms/step - loss: 0.5569 - rmse: 0.4921 - acc: 0.9953 - val_loss: 0.6132 - val_rmse: 0.3140 - val_acc: 0.9991

Epoch 00001: val_loss improved from inf to 0.61316, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99535, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.by.accuracy.hdf5
Epoch 2/500
16330/16330 [==============================] - 311s 19ms/step - loss: 0.5003 - rmse: 0.4112 - acc: 0.9994 - val_loss: 0.5889 - val_rmse: 0.2837 - val_acc: 0.9991

Epoch 00002: val_loss improved from 0.61316 to 0.58892, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.hdf5

Epoch 00002: acc improved from 0.99535 to 0.99939, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.bes

Epoch 26/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2982 - rmse: 0.1113 - acc: 0.9988 - val_loss: 0.5107 - val_rmse: 0.1479 - val_acc: 0.9990

Epoch 00026: val_loss did not improve from 0.50933

Epoch 00026: acc did not improve from 0.99939
Epoch 27/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.3044 - rmse: 0.1101 - acc: 0.9988 - val_loss: 0.5106 - val_rmse: 0.1482 - val_acc: 0.9991

Epoch 00027: val_loss did not improve from 0.50933

Epoch 00027: acc did not improve from 0.99939
Epoch 28/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.3025 - rmse: 0.1060 - acc: 0.9988 - val_loss: 0.5115 - val_rmse: 0.1528 - val_acc: 0.9990

Epoch 00028: val_loss did not improve from 0.50933

Epoch 00028: acc did not improve from 0.99939
Epoch 29/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2986 - rmse: 0.1050 - acc: 0.9987 - val_loss: 0.5107 - val_rmse: 0.1479 - val_acc: 0.9990

Ep

16330/16330 [==============================] - 306s 19ms/step - loss: 0.2433 - rmse: 0.0758 - acc: 0.9988 - val_loss: 0.5449 - val_rmse: 0.1552 - val_acc: 0.9990

Epoch 00056: val_loss did not improve from 0.50933

Epoch 00056: acc did not improve from 0.99939
Epoch 57/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2315 - rmse: 0.0770 - acc: 0.9988 - val_loss: 0.5497 - val_rmse: 0.1526 - val_acc: 0.9990

Epoch 00057: val_loss did not improve from 0.50933

Epoch 00057: acc did not improve from 0.99939
Epoch 58/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2335 - rmse: 0.0773 - acc: 0.9991 - val_loss: 0.5875 - val_rmse: 0.1814 - val_acc: 0.9978

Epoch 00058: val_loss did not improve from 0.50933

Epoch 00058: acc did not improve from 0.99939
Epoch 59/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2352 - rmse: 0.0846 - acc: 0.9985 - val_loss: 0.5538 - val_rmse: 0.1520 - val_acc: 0.9990

Epoch 00059: va

KeyboardInterrupt: 

## Inferência

In [45]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


In [46]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Inserir dropout

In [50]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)

activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dropout = Dropout(0.5)(activation2)

dense_layer2 = Dense(32)(dropout)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
dropout1 = Dropout(0.5)(activation3)

if(flag_6_outputs):
    output = Dense(6, name='predictions')(dropout1)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(dropout1)

model = Model([input_a, input_b], output)

learning_rate = 0.00001

def vo_loss_mod_thesis(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

num_epochs = 100

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod_thesis], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_52 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_26 (Sequential)      (None, 50688)        1918848     input_51[0][0]                   
                                                                 input_52[0][0]                   
__________________________________________________________________________________________________
lambda_24 (Lambda)              (None, 50688)        0           sequential_26[1][0]              
          

### Preparando os checkpoints

In [51]:
model_path = "D:/KITTI/Graph/DepthVOExpanded_Dropout/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.val_loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVOExpanded_Dropout/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVOExpanded_Dropout/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/100
16330/16330 [==============================] - 317s 19ms/step - loss: 0.6061 - rmse: 0.5903 - acc: 0.9936 - val_loss: 0.6054 - val_rmse: 0.3153 - val_acc: 0.9991

Epoch 00001: val_loss improved from inf to 0.60544, saving model to D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.val_loss.hdf5

Epoch 00001: acc improved from -inf to 0.99357, saving model to D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.by.accuracy.hdf5
Epoch 2/100
16330/16330 [==============================] - 305s 19ms/step - loss: 0.5690 - rmse: 0.5251 - acc: 0.9994 - val_loss: 0.5870 - val_rmse: 0.2839 - val_acc: 0.9991

Epoch 00002: val_loss improved from 0.60544 to 0.58704, saving model to D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.val_loss.hdf5

Epoch 00002: acc improved from 0.99357 to 0.99939, saving model to D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.by.ac


Epoch 00025: val_loss did not improve from 0.51045

Epoch 00025: acc did not improve from 0.99939
Epoch 26/100
16330/16330 [==============================] - 305s 19ms/step - loss: 0.3796 - rmse: 0.2244 - acc: 0.9990 - val_loss: 0.5124 - val_rmse: 0.1585 - val_acc: 0.9991

Epoch 00026: val_loss did not improve from 0.51045

Epoch 00026: acc did not improve from 0.99939
Epoch 27/100
16330/16330 [==============================] - 305s 19ms/step - loss: 0.3772 - rmse: 0.2224 - acc: 0.9993 - val_loss: 0.5057 - val_rmse: 0.1511 - val_acc: 0.9991

Epoch 00027: val_loss improved from 0.51045 to 0.50570, saving model to D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.val_loss.hdf5

Epoch 00027: acc did not improve from 0.99939
Epoch 28/100
16330/16330 [==============================] - 305s 19ms/step - loss: 0.3760 - rmse: 0.2195 - acc: 0.9988 - val_loss: 0.5112 - val_rmse: 0.1555 - val_acc: 0.9991

Epoch 00028: val_loss did not improve from 0.50570

Epoch 00028: acc did not improve


Epoch 00054: val_loss did not improve from 0.50380

Epoch 00054: acc did not improve from 0.99939
Epoch 55/100
16330/16330 [==============================] - 312s 19ms/step - loss: 0.3260 - rmse: 0.1730 - acc: 0.9985 - val_loss: 0.5088 - val_rmse: 0.1493 - val_acc: 0.9991

Epoch 00055: val_loss did not improve from 0.50380

Epoch 00055: acc did not improve from 0.99939
Epoch 56/100
16330/16330 [==============================] - 326s 20ms/step - loss: 0.3410 - rmse: 0.1745 - acc: 0.9985 - val_loss: 0.5085 - val_rmse: 0.1487 - val_acc: 0.9991

Epoch 00056: val_loss did not improve from 0.50380

Epoch 00056: acc did not improve from 0.99939
Epoch 57/100
16330/16330 [==============================] - 314s 19ms/step - loss: 0.3393 - rmse: 0.1727 - acc: 0.9981 - val_loss: 0.5169 - val_rmse: 0.1551 - val_acc: 0.9991

Epoch 00057: val_loss did not improve from 0.50380

Epoch 00057: acc did not improve from 0.99939
Epoch 58/100
16330/16330 [==============================] - 314s 19ms/step - lo

KeyboardInterrupt: 

## Inferência

In [52]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Dropout/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Dropout/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


In [53]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Dropout/model/weights.best.val_loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Dropout/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Dropout/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Mudar parâmetro beta

In [55]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dense_layer2 = Dense(32)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

learning_rate = 0.00001

alpha  = 1
beta = 300

def vo_loss_mod_thesis(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    alpha
    mean_rot = mean[:, 0] * beta
    mean_trasl = mean[:, 1]* alpha   
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

def mean_loss(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_trasl = mean[:, 3:6]
    mean_rot = mean[:, 0:3]
    #mean = K.concatenate([mean_rot, mean_trasl])
    sqrt_trasl = K.sqrt(K.mean(mean_rot, axis=-1))
    sqrt_rot = K.sqrt(K.mean(mean_trasl, axis=-1))
    return (sqrt_rot/sqrt_trasl)

num_epochs = 50

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod_thesis], optimizer=opt, metrics=[rmse, mean_loss, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_55 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_56 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_28 (Sequential)      (None, 50688)        1918848     input_55[0][0]                   
                                                                 input_56[0][0]                   
__________________________________________________________________________________________________
lambda_26 (Lambda)              (None, 50688)        0           sequential_28[1][0]              
          

### Preparando os checkpoints

In [56]:
model_path = "D:/KITTI/Graph/DepthVOExpanded_beta_300/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVOExpanded_beta_300/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVOExpanded_beta_300/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/50
16330/16330 [==============================] - 311s 19ms/step - loss: 0.8926 - rmse: 0.4890 - mean_loss: nan - acc: 0.9988 - val_loss: 0.9369 - val_rmse: 0.3245 - val_mean_loss: nan - val_acc: 0.9991

Epoch 00001: val_loss improved from inf to 0.93686, saving model to D:/KITTI/Graph/DepthVOExpanded_beta_300/model/weights.best.by.val.loss.hdf5

Epoch 00001: acc improved from -inf to 0.99884, saving model to D:/KITTI/Graph/DepthVOExpanded_beta_300/model/weights.best.by.accuracy.hdf5
Epoch 2/50
16330/16330 [==============================] - 314s 19ms/step - loss: 0.7857 - rmse: 0.4035 - mean_loss: nan - acc: 0.9994 - val_loss: 0.8961 - val_rmse: 0.2938 - val_mean_loss: nan - val_acc: 0.9991

Epoch 00002: val_loss improved from 0.93686 to 0.89610, saving model to D:/KITTI/Graph/DepthVOExpanded_beta_300/model/weights.best.by.val.loss.hdf5

Epoch 00002: acc improved from 0.99884 to 0.99

16330/16330 [==============================] - 313s 19ms/step - loss: 0.3857 - rmse: 0.0765 - mean_loss: nan - acc: 0.9991 - val_loss: 0.7800 - val_rmse: 0.1520 - val_mean_loss: nan - val_acc: 0.9990

Epoch 00048: val_loss did not improve from 0.74309

Epoch 00048: acc did not improve from 0.99939
Epoch 49/50
16330/16330 [==============================] - 312s 19ms/step - loss: 0.3715 - rmse: 0.0747 - mean_loss: nan - acc: 0.9991 - val_loss: 0.7757 - val_rmse: 0.1470 - val_mean_loss: nan - val_acc: 0.9990

Epoch 00049: val_loss did not improve from 0.74309

Epoch 00049: acc did not improve from 0.99939
Epoch 50/50
16330/16330 [==============================] - 308s 19ms/step - loss: 0.3748 - rmse: 0.0750 - mean_loss: nan - acc: 0.9989 - val_loss: 0.7844 - val_rmse: 0.1453 - val_mean_loss: nan - val_acc: 0.9988

Epoch 00050: val_loss did not improve from 0.74309

Epoch 00050: acc did not improve from 0.99939


## Inferência

In [57]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_beta_300/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_beta_300/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_beta_300/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


In [58]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_beta_300/model/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_beta_300/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_beta_300/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


## Read and transform training pose (of thesis) with Ty

In [14]:
cont = 0
label_training_thesis_0 = {}
label_training_thesis_1 = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   

        label_aux.append(matrix)
        
        
    label_training_thesis_0[name] = label_aux[:-1]
    label_training_thesis_1[name] = label_aux[1:]


## Transformation

In [15]:
from math import asin, atan2, sqrt

T_train_thesis = {}

for directory in training_seqs:    
    curr_sequence = training_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_training_thesis_0[name], label_training_thesis_1[name]):
        
        Tx = pose_1[0][3]
        Ty = pose_1[1][3]
        Tz = pose_1[2][3]
        
        Tx_ant = pose_0[0][3]
        Ty_ant = pose_0[1][3]
        Tz_ant = pose_0[2][3]

        Ri_1 = pose_1[0][0]
        Ri_3 = pose_1[0][2]    
        Ri_1_ant = pose_0[0][0]
        Ri_3_ant = pose_0[0][2]
        
        Ti = [Tx, Ty, Tz]
        Ti_ant = [Tx_ant, Ty_ant, Tz_ant]

        displacement = sqrt(sum([(a - b) ** 2 for a, b in zip(Ti, Ti_ant)]))
        delta_Theta = atan2(Ri_3, Ri_1) - atan2(Ri_3_ant, Ri_1_ant)
    
        T_aux.append([delta_Theta, displacement])
    
    T_train_thesis[name] = T_aux
    print(len(T_train_thesis[name]))
    

4540
1100
4660
800
270
2760
1100
1100


## Read and transform test pose (of thesis)

In [16]:
cont = 0
label_test_thesis_0 = {}
label_test_thesis_1 = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    labels = curr_sequence.get_labels()
    name = curr_sequence.get_dir()
    label_aux = []
    
    for label in labels:
        matrix = np.reshape(label, (3, 4))
        homogeneous = np.array([[0, 0, 0, 1]])
        matrix = np.vstack((matrix, homogeneous))   

        label_aux.append(matrix)
        
        
    label_test_thesis_0[name] = label_aux[:-1]
    label_test_thesis_1[name] = label_aux[1:]


## Transformation

In [18]:
T_test_thesis = {}

for directory in test_seqs:    
    curr_sequence = test_seqs[directory]
    name = curr_sequence.get_dir()
    T_aux = []
    
    for pose_0, pose_1 in zip(label_test_thesis_0[name], label_test_thesis_1[name]):
        
        Tx = pose_1[0][3]
        Ty = pose_1[1][3]
        Tz = pose_1[2][3]
        
        Tx_ant = pose_0[0][3]
        Ty_ant = pose_0[1][3]
        Tz_ant = pose_0[2][3]

        Ri_1 = pose_1[0][0]
        Ri_3 = pose_1[0][2]    
        Ri_1_ant = pose_0[0][0]
        Ri_3_ant = pose_0[0][2]
        
        Ti = [Tx, Ty, Tz]
        Ti_ant = [Tx_ant, Ty_ant, Tz_ant]

        displacement = sqrt(sum([(a - b) ** 2 for a, b in zip(Ti, Ti_ant)]))
        delta_Theta = atan2(Ri_3, Ri_1) - atan2(Ri_1_ant, Ri_3_ant)
    
        T_aux.append([delta_Theta, displacement])
    
    T_test_thesis[name] = T_aux
    # print(T_train)
    

In [19]:
training_poses_transformed_thesis = []

for sequence in kitti_train_dirs:
  
    for pose in T_train_thesis[sequence]:
        training_poses_transformed_thesis.append(pose)

In [20]:
test_poses_transformed_thesis = []

for sequence in kitti_test_dirs:    
        for pose in T_test_thesis[sequence]:
            test_poses_transformed_thesis.append(pose)
            

### Preparando a entrada para o padrão Tensorflow

In [25]:
training_r = np.reshape(training_r, (len(training_r), input_height, input_width, 1))
training_l = np.reshape(training_l, (len(training_l), input_height, input_width, 1))

print(np.shape(training_r))

test_r = np.reshape(test_r, (len(test_r), input_height, input_width, 1))
test_l = np.reshape(test_l, (len(test_l), input_height, input_width, 1))
print(np.shape(test_r))

(16330, 47, 155, 1)
(6860, 47, 155, 1)


In [21]:
y_training = np.reshape(training_poses_transformed_thesis, (len(training_poses_transformed_thesis), 2))
y_test = np.reshape(test_poses_transformed_thesis, (len(test_poses_transformed_thesis), 2))
print(np.shape(y_training))

(16330, 2)


# Definindo a arquitetura
***

### Usando Leaky Relu

* Dados originais

In [23]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer)
dense_layer2 = Dense(32)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

learning_rate = 0.00001
num_epochs = 50

def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt


opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 50688)        1918848     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 50688)        0           sequential_2[1][0]               
          

### Preparando os checkpoints

In [ ]:
model_path = "D:/KITTI/Graph/DepthVO_with_Ty/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_with_Ty/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/50
16330/16330 [==============================] - 319s 20ms/step - loss: 0.5986 - rmse: 0.5444 - acc: 0.9892 - val_loss: 13.8803 - val_rmse: 1.2114 - val_acc: 0.5602

Epoch 00001: val_loss improved from inf to 13.88031, saving model to D:/KITTI/Graph/DepthVO_with_Ty/model/weights.best.val.loss.hdf5

Epoch 00001: acc improved from -inf to 0.98916, saving model to D:/KITTI/Graph/DepthVO_with_Ty/model/weights.best.by.accuracy.hdf5
Epoch 2/50
16330/16330 [==============================] - 313s 19ms/step - loss: 0.5358 - rmse: 0.4648 - acc: 0.9994 - val_loss: 13.8623 - val_rmse: 1.2013 - val_acc: 0.5602

Epoch 00002: val_loss improved from 13.88031 to 13.86229, saving model to D:/KITTI/Graph/DepthVO_with_Ty/model/weights.best.val.loss.hdf5

Epoch 00002: acc improved from 0.98916 to 0.99939, saving model to D:/KITTI/Graph/DepthVO_with_Ty/model/weights.best.by.accuracy.hdf5
Epoch 3/50
 9392/16330 [================>.............] - ETA: 

## Inferência

In [25]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_RELU_thesis_expanded_network/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-1-expanded/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-1-expanded/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Inserindo recorrência

In [47]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Reshape 
from keras.layers import Conv2D, add, concatenate, LeakyReLU, LSTM, CuDNNGRU, Bidirectional, TimeDistributed
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)

activation1 = LeakyReLU(alpha=0.2)(dense_layer)

dense_layer1 = TimeDistributed(Dense(32))(activation1)
recurrency = CuDNNGRU(512)(dense_layer1)

activation2 = LeakyReLU(alpha=0.1)(recurrency)

#modifiquei a entrada aqui estava errado


if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation2)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation2)

model = Model([input_a, input_b], output)

learning_rate = 0.00001

def vo_loss_mod_thesis(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt

num_epochs = 500

opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss_mod_thesis], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


IndexError: list index out of range

### Preparando os checkpoints

In [44]:
model_path = "D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/1/', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=num_epochs, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/
Train on 16330 samples, validate on 6860 samples
Epoch 1/500
16330/16330 [==============================] - 310s 19ms/step - loss: 0.5569 - rmse: 0.4921 - acc: 0.9953 - val_loss: 0.6132 - val_rmse: 0.3140 - val_acc: 0.9991

Epoch 00001: val_loss improved from inf to 0.61316, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99535, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.by.accuracy.hdf5
Epoch 2/500
16330/16330 [==============================] - 311s 19ms/step - loss: 0.5003 - rmse: 0.4112 - acc: 0.9994 - val_loss: 0.5889 - val_rmse: 0.2837 - val_acc: 0.9991

Epoch 00002: val_loss improved from 0.61316 to 0.58892, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.hdf5

Epoch 00002: acc improved from 0.99535 to 0.99939, saving model to D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.bes

Epoch 26/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2982 - rmse: 0.1113 - acc: 0.9988 - val_loss: 0.5107 - val_rmse: 0.1479 - val_acc: 0.9990

Epoch 00026: val_loss did not improve from 0.50933

Epoch 00026: acc did not improve from 0.99939
Epoch 27/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.3044 - rmse: 0.1101 - acc: 0.9988 - val_loss: 0.5106 - val_rmse: 0.1482 - val_acc: 0.9991

Epoch 00027: val_loss did not improve from 0.50933

Epoch 00027: acc did not improve from 0.99939
Epoch 28/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.3025 - rmse: 0.1060 - acc: 0.9988 - val_loss: 0.5115 - val_rmse: 0.1528 - val_acc: 0.9990

Epoch 00028: val_loss did not improve from 0.50933

Epoch 00028: acc did not improve from 0.99939
Epoch 29/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2986 - rmse: 0.1050 - acc: 0.9987 - val_loss: 0.5107 - val_rmse: 0.1479 - val_acc: 0.9990

Ep

16330/16330 [==============================] - 306s 19ms/step - loss: 0.2433 - rmse: 0.0758 - acc: 0.9988 - val_loss: 0.5449 - val_rmse: 0.1552 - val_acc: 0.9990

Epoch 00056: val_loss did not improve from 0.50933

Epoch 00056: acc did not improve from 0.99939
Epoch 57/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2315 - rmse: 0.0770 - acc: 0.9988 - val_loss: 0.5497 - val_rmse: 0.1526 - val_acc: 0.9990

Epoch 00057: val_loss did not improve from 0.50933

Epoch 00057: acc did not improve from 0.99939
Epoch 58/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2335 - rmse: 0.0773 - acc: 0.9991 - val_loss: 0.5875 - val_rmse: 0.1814 - val_acc: 0.9978

Epoch 00058: val_loss did not improve from 0.50933

Epoch 00058: acc did not improve from 0.99939
Epoch 59/500
16330/16330 [==============================] - 306s 19ms/step - loss: 0.2352 - rmse: 0.0846 - acc: 0.9985 - val_loss: 0.5538 - val_rmse: 0.1520 - val_acc: 0.9990

Epoch 00059: va

KeyboardInterrupt: 

## Inferência

In [45]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


In [46]:
from math import degrees, sin, cos, pi

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVOExpanded_Mod_Loss_500/model/weights.best.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVOExpanded_Mod_Loss_500/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:




### Preparando os checkpoints e treinando

In [18]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(1028, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
#activation2 = Activation('relu')(dense_layer1)

activation2 = LeakyReLU(alpha=0.2)(abs_diff)
dense_layer2 = Dense(1024)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 1028)         9041412     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1028)         0           sequential_2[1][0]               
          

### Preparando os checkpoints

In [121]:
model_path = "D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_Fully_Connected_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 32s 2ms/step - loss: 0.1776 - rmse: 0.3841 - acc: 0.9955 - val_loss: 0.0870 - val_rmse: 0.2368 - val_acc: 0.9984

Epoch 00001: val_loss improved from inf to 0.08696, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.99553, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 32s 2ms/step - loss: 0.0692 - rmse: 0.2293 - acc: 0.9985 - val_loss: 0.0813 - val_rmse: 0.2209 - val_acc: 0.9988

Epoch 00002: val_loss improved from 0.08696 to 0.08128, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.99553 to 0.99847, saving model to D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5
Epoch 3/25
16330/16330 [======

## Inferência

In [19]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-2/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-2/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Definindo a arquitetura
***

### Usando concatenate no lugar de get_abs_diff

* Dados originais

### Preparando os checkpoints e treinando

In [20]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(1028, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
dense_layer = Dense(1024)(concatenetion_layer)
activation1 = LeakyReLU(alpha=0.5)(dense_layer)
dense_layer1 = Dense(1024)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 1028)         9041412     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 2056)         0           sequential_3[1][0]               
          

### Preparando os checkpoints

In [125]:
model_path = "D:/KITTI/Graph/DepthVO_concatenate_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_concatenate_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 39s 2ms/step - loss: 0.2516 - rmse: 0.2567 - acc: 0.9885 - val_loss: 0.0831 - val_rmse: 0.2074 - val_acc: 0.9980

Epoch 00001: val_loss improved from inf to 0.08315, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.98855, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 39s 2ms/step - loss: 0.0459 - rmse: 0.1678 - acc: 0.9968 - val_loss: 0.0725 - val_rmse: 0.1872 - val_acc: 0.9988

Epoch 00002: val_loss improved from 0.08315 to 0.07247, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.98855 to 0.99675, saving model to D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.by.accuracy.hdf5
Epoch 3/25
16330/16330 [======================

## Inferência

In [21]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_concatenate_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-3/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-3/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:




### Preparando os checkpoints e treinando

In [22]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = False
flag_3_outputs = True

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(512, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(1028, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
activation2 = LeakyReLU(alpha=0.2)(concatenetion_layer)
dense_layer2 = Dense(1024)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 1028)         9041412     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 2056)         0           sequential_4[1][0]               
          

### Preparando os checkpoints

In [29]:
model_path = "D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
16330/16330 [==============================] - 32s 2ms/step - loss: 0.1909 - rmse: 0.2796 - acc: 0.9873 - val_loss: 0.0820 - val_rmse: 0.2205 - val_acc: 0.9985

Epoch 00001: val_loss improved from inf to 0.08203, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00001: acc improved from -inf to 0.98732, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5
Epoch 2/25
16330/16330 [==============================] - 30s 2ms/step - loss: 0.0532 - rmse: 0.1898 - acc: 0.9976 - val_loss: 0.0817 - val_rmse: 0.2075 - val_acc: 0.9990

Epoch 00002: val_loss improved from 0.08203 to 0.08175, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.hdf5

Epoch 00002: acc improved from 0.98732 to 0.99755, saving model to D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best

## Inferência

In [23]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_concatenate_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/thesis-4/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/thesis-4/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

Sequence 00:
Sequence 01:
Sequence 02:
Sequence 03:
Sequence 04:
Sequence 05:
Sequence 06:
Sequence 07:
Sequence 08:
Sequence 09:
Sequence 10:


# Usando as 6-DOF

* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:




### Preparando os checkpoints e treinando

In [29]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())
    #seq.add(Dense(1028))
    #seq.add(Activation('relu'))
    # seq.add(Dropout(0.5))
    # seq.add(Dense(32))

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

abs_diff = Lambda(get_abs_diff, output_shape=abs_diff_output_shape)([processed_a, processed_b])
dense_layer = Dense(64)(abs_diff)
#activation1 = LeakyReLU(alpha=0.5)(dense_layer)
#dense_layer1 = Dense(64)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer)
dense_layer2 = Dense(32)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

learning_rate = 0.0001
num_epochs = 50

def vo_loss_mod(y_true, y_pred):
    mean = K.square(y_pred - y_true)
    mean_rot = mean[:, 0] * 150
    mean_trasl = mean[:, 1]* 0.3    
    mean = K.concatenate([mean_rot, mean_trasl])
    sqrt = K.sqrt(K.mean(mean, axis=-1))
    return sqrt


opt = optm.Adam(lr=learning_rate)
model.compile(loss=[vo_loss], optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 50688)        1918848     input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 50688)        0           sequential_5[1][0]               
          

### Preparando os checkpoints

In [30]:
model_path = "D:/KITTI/Graph/DepthVO_6DOF/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_6DOF', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

Train on 16330 samples, validate on 6860 samples
Epoch 1/25
 1088/16330 [>.............................] - ETA: 4:14 - loss: 1.7629 - rmse: 3.2127 - acc: 0.1057

KeyboardInterrupt: 

## Inferência

In [ ]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_6DOF/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

In [ ]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_6DOF/model/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF/val_loss%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

# Definindo a arquitetura
***

### Usando concatenate no lugar de get_abs_diff

* Dados originais

### Preparando os checkpoints e treinando

In [32]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

    seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
dense_layer = Dense(1024)(concatenetion_layer)
activation1 = LeakyReLU(alpha=0.5)(dense_layer)
dense_layer1 = Dense(1024)(activation1)
activation2 = LeakyReLU(alpha=0.2)(dense_layer1)
dense_layer2 = Dense(64)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 47, 155, 1)   0                                            
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 50688)        1918848     input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 101376)       0           sequential_6[1][0]               
          

### Preparando os checkpoints

In [ ]:
model_path = "D:/KITTI/Graph/DepthVO_6DOF_concatenate_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_6DOF_concatenate_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

## Inferência

In [ ]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_6DOF_concatenate_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_thesis/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_thesis/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

In [ ]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_6DOF_concatenate_thesis/model/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_thesis/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_thesis/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

* Mesmos resultados que com a LeakyRelu

## Diminuindo a quantidade de Fully connect layers:




### Preparando os checkpoints e treinando

In [ ]:
from objectives.VoObjectives import vo_loss, rmse

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, MaxPooling2D, merge, Conv2D, add, concatenate, LeakyReLU
from keras.layers.core import Activation, Flatten
from keras.optimizers import RMSprop
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras import optimizers as optm
from keras import losses
from keras.callbacks import TensorBoard, ModelCheckpoint

flag_6_outputs = True
flag_3_outputs = False

learning_rate = 0.0001
epochs = 200

def get_abs_diff(vects):
    x, y = vects
    return K.abs(x - y)

def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1

def create_cnn_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''

        seq = Sequential()
    seq.add(Conv2D(64, (3, 3), input_shape=input_dim))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Conv2D(64, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(128, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    #seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    
    seq.add(Conv2D(256, (3, 3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))

    seq.add(Flatten())

    return seq

input_shape = (input_height, input_width, 1)
base_network = create_cnn_network(input_shape)

input_a = Input(input_shape)
input_b = Input(input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

concatenetion_layer = concatenate([processed_a, processed_b])
#dense_layer = Dense(1028)(abs_diff)
#activation1 = Activation('relu')(dense_layer)
#dense_layer1 = Dense(1028)(activation1)
activation2 = LeakyReLU(alpha=0.2)(concatenetion_layer)
dense_layer2 = Dense(1024)(activation2)
activation3 = LeakyReLU(alpha=0.1)(dense_layer2)
    
if(flag_6_outputs):
    output = Dense(6, name='predictions')(activation3)
    
elif(flag_3_outputs):
    output = Dense(2, name='predictions')(activation3)

model = Model([input_a, input_b], output)

opt = optm.Adam(lr=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse, 'acc'])
model.summary()


### Preparando os checkpoints

In [ ]:
model_path = "D:/KITTI/Graph/DepthVO_6DOF_concatenate_Fully_Connected_thesis/model/"
if not os.path.exists(os.path.dirname(model_path)):
        print (model_path)
        os.makedirs(os.path.dirname(model_path))

filepath=model_path+"weights.best.by.val.loss.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

tbCallBack = TensorBoard(log_dir='D:/KITTI/Graph/DepthVO_6DOF_concatenate_Fully_Connected_thesis', histogram_freq=0, write_graph=True, 
                         write_images=True)

filepath2=model_path+"weights.best.by.accuracy.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint, checkpoint2, tbCallBack]

model.fit([training_r, training_l], y_training, batch_size=16, epochs=25, 
                             validation_data=([test_r, test_l], y_test), callbacks=callbacks_list)

## Inferência

In [ ]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_6DOF_concatenate_Fully_Connected_thesis/model/weights.best.by.accuracy.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_Fully_Connected_thesis/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_Fully_Connected_thesis/acc/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')

In [ ]:
from math import degrees, sin, cos

for sequence in kitti_train_dirs:
    
    print("Sequence %s:"%(sequence))
    cont = 1
    data_training_r = np.array(dataset_training_all_r[sequence])
    data_training_l = np.array(dataset_training_all_l[sequence]) 
    
   
    model.load_weights(filepath='D:/KITTI/Graph/DepthVO_6DOF_concatenate_Fully_Connected_thesis/model/weights.best.by.val.loss.hdf5')    
    prediction = model.predict([data_training_r.reshape((len(data_training_r), input_height, input_width, 1)), 
                               data_training_l.reshape((len(data_training_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_training_r)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
    
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_Fully_Connected_thesis/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')
    
for sequence in kitti_test_dirs:
    cont = 1
    print("Sequence %s:"%(sequence))
    data_test_r = np.array(dataset_test_all_r[sequence])
    data_test_l = np.array(dataset_test_all_l[sequence])
    
    prediction = model.predict([data_test_r.reshape((len(data_test_r), input_height, input_width, 1)), 
                               data_test_l.reshape((len(data_test_l), input_height, input_width, 1))])
    
    sequence_GT = np.zeros((len(data_test_l)+1, 12))
    
    init_mat = np.identity(4)
    sequence_GT[0, :] = init_mat[0:3,:].flatten()    
    
    position_X = 0
    position_Z = 0
    angle_total = pi/2
    
    for element in prediction:
        angle = element[0]
        displacement = element[1]
        
        angle_total = angle_total - angle
        
        position_X = position_X + displacement*cos(angle_total)
        position_Z = position_Z + displacement*sin(angle_total)
        
        sequence_GT[cont, :] = init_mat[0:3,:].flatten()
        
        sequence_GT[cont, 0] = cos(angle_total-(pi/2))
        sequence_GT[cont, 2] = -sin(angle_total-(pi/2))
        sequence_GT[cont, 3] = position_X
        sequence_GT[cont, 8] = sin(angle_total-(pi/2))
        sequence_GT[cont, 10] = cos(angle_total-(pi/2))
        sequence_GT[cont, 11] = position_Z
        
        cont += 1
        
    np.savetxt("D:/KITTI/output_to_draw/DepthVO_6DOF_concatenate_Fully_Connected_thesis/val_loss/%s.txt"%(sequence), np.array(sequence_GT), delimiter=" ", fmt='%s')